In [468]:
import pandas as pd 
from datetime import datetime
import openpyxl as px 
import psycopg2 as pg
import configparser
import numpy as np
import os

In [469]:
today = pd.Timestamp(datetime.today())

In [470]:
# Configurar a opção para evitar downcasting silencioso
pd.set_option('future.no_silent_downcasting', True)

In [471]:
hora_inicio = datetime.today()

In [472]:
crendenciais_path = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\18 - SQL\sets.conf'

In [473]:
#Senhas e credênciais 

credenciais = configparser.ConfigParser()

credenciais.read(crendenciais_path)

logins = credenciais['informacoes']


conn_dwh = pg.connect(
    user =logins['login'],
    password =logins['senha'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
)

In [474]:
def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [475]:

#Realizando conexão do banco -> cadastro histórico

cursor = conn_dwh.cursor()

#Se atentar a região de São paulo que é 1 

#Necessário mudar a consulta para fazer um left join na consulta d_cfg_espacamento para trazer a informação de espaçamento do cadastro e nome do projeto, logo foi necessário ter duas consultas


cursor.execute("""
SELECT 
    f.data_plantio,
    f.cd_regiao,
    f.id_projeto,
    f.cd_projeto,
    f.cd_talhao,
    f.num_ciclo,
    f.cd_material_genetico,
    f.cd_regime,
    f.num_rotacao,
    f.vlr_area,
    f.source,
    f.data_reg,
    f.est_reg,
    f.cd_uso_solo,
    f.cd_caracteristica_uso_solo,
    f.cd_espacamento,
    d.dcr_espacamento,
    p.nom_projeto
FROM 
    forestry.f_cfg_uso_solo f
LEFT JOIN 
    forestry.d_cfg_espacamento d
ON 
    f.cd_espacamento = d.cd_espacamento
    
LEFT JOIN 
    forestry.cfg_projeto p

ON
    f.id_projeto = p.id_projeto
    
WHERE 
    f.source = 'BSP' 
    AND p.source = 'BSP'
    AND p.cd_regiao = '1'
    AND p.est_reg = 'A'
    AND f.tip_reg = 'A'  
    AND f.cd_regiao IN ('1','2')
    
    
    
""")



dados_consulta = cursor.fetchall()

cursor.close()

#Manter a conexão do dwh aberta
#conn_dwh.close() A conexão não foi fechada pois será utilizada novamente na consulta abaixo

colunas_cad = [desc[0] for desc in cursor.description]


cadastro_ciclo = pd.DataFrame(dados_consulta,columns=colunas_cad)

In [476]:
#Base de Material genético

cursor_mg = conn_dwh.cursor()

cursor_mg.execute(
    """
    SELECT DISTINCT ON (cd_material_genetico)

    source,
    species,
    cd_material_genetico,
    dcr_material_genetico,
    abrev_material_genetico,
    tipo_propriedade

FROM 
    forestry.f_sgf_lb_con_base_dim

WHERE 

    source = 'BSP'
    and cd_material_genetico is not null
    
    """
)


dados_mg = cursor_mg.fetchall()

cursor_mg.close()

#Manter a conexão do dwh aberta
#conn_dwh.close()

colunas_mg = [desc[0] for desc in cursor_mg.description]

material_genetico = pd.DataFrame(dados_mg, columns=colunas_mg)


In [477]:
#Base de CTO de baldeio 

cursor_cto = conn_dwh.cursor()

cursor_cto.execute(

"""
SELECT
    cd_uso_solo,
    id_projeto,
    cd_talhao,
    dcr_produto,
    dcr_operacao,
    data_cto,
    flag_cto_executado,
    source
    

FROM 
    forestry.vw_f_vw_cubo_col_talhoes_cto_bsp_all

where 
    source = 'BSP' and
    dcr_operacao in( 'Baldeio Forwarder CEL', 'Baldeio Forwarder ENE')

order by
    cd_uso_solo ASC,
    data_cto DESC

"""

)

dados_cto_baldeio  = cursor_cto.fetchall()

cursor_cto.close()

#Fechando conexão com o DWH
conn_dwh.close()


colunas_cto = [desc[0] for desc in cursor_cto.description]

cto_baldeio = pd.DataFrame(dados_cto_baldeio, columns=colunas_cto)







In [478]:
df_boletim1 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2020_Boletim_da_Silvicultura.parquet'
df_boletim2 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2021_Boletim_da_Silvicultura.parquet'
df_boletim3 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2022_Boletim_da_Silvicultura.parquet'
df_boletim4 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2023_Boletim_da_Silvicultura.parquet'
df_boletim5 =r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2024_Boletim_da_Silvicultura.parquet'

In [479]:
path_premissas = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\premissas.xlsx'

In [480]:
path_justificativas = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\justificativas.xlsx'

In [481]:
path_recomendacao_adu = r'F:\Silvicultura\RECOMENDAÇÕES DE ADUBAÇÃO\RECOMENDAÇÕES_ADUBAÇÃO_SILVICULTURA_SP.xlsx'

path_rec_complementar = r'F:\Silvicultura\RECOMENDAÇÕES DE ADUBAÇÃO\RECOMENDAÇÕES_ADUBAÇÕES_COMPLEMENTARES_SP.xlsx'


In [482]:
path_retalhonamento = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento_e_Controle\Geoprocessamento\02_SETOR\07_CADASTRO\14_BASE RETALHONAMENTOS\DExPARA\SHP'

In [483]:
path_topografia = r'\\Glwfs02\lwc-florestal\Planejamento_e_Controle\Geoprocessamento\02_SETOR\XX_GIS\04_ATUALIZAÇÃO_BASE\FUP_GIS_Controle_ajuste_de_base_Oficial.xlsx'

In [484]:
path_micro = r'\\Glwfs02\lwc-florestal\Planejamento_Florestal\Macro-e-Microplanejamento\A.2.SILVICULTURA - FUP - Silvicultura - Oficial.xlsb'

In [485]:
path_pas = 'F:\Planejamento_Florestal\Equipe\Pedro de Oliveira Sousa Castro\Silvicultura\Automação Plano de Demandas\Bases\PAS.xlsx'

In [486]:
#Base Micro 

#Data Base excel

data_base_excel = pd.Timestamp('1899-12-30')

base_micro = pd.read_excel(path_micro,sheet_name='COMPILAÇÃO',skiprows=1, usecols=[' INDEX','Divulgação_Book','Mapa_Micro'], dtype={' INDEX':'object'})

#Transformando linhas que temos '-' para Nan utilizando o numpy
base_micro['Mapa_Micro'] = base_micro['Mapa_Micro'].replace({'-': np.nan})

base_micro['Divulgação_Book'] = base_micro['Divulgação_Book'].replace({'-':np.nan})


#Foi necessário alterar o tipo de data para inteiro, e depois somar a data base do excel devido ao formato da base
base_micro['Divulgação_Book'] = pd.to_numeric(base_micro['Divulgação_Book'], errors='coerce').astype('Int32')
base_micro['Mapa_Micro'] = pd.to_numeric(base_micro['Mapa_Micro'], errors='coerce').astype('Int32')

base_micro['Divulgação_Book'] = base_micro['Divulgação_Book'].apply(
    lambda x: data_base_excel + pd.Timedelta(days=x) if pd.notna(x) else np.nan
)

base_micro['Mapa_Micro'] = base_micro['Mapa_Micro'].apply(
    lambda x: data_base_excel + pd.Timedelta(days=x) if pd.notna(x) else np.nan
)


#Removendo linhas vazias dentro da base do micro
base_micro = base_micro[pd.notna(base_micro[' INDEX'])]
base_micro = base_micro.drop_duplicates([' INDEX'])

#Removendo as linhas onde temos todas as linhas nulas em ambas as colunas
base_micro = base_micro[~ base_micro[['Divulgação_Book','Mapa_Micro']].isna().all(axis=1)]



In [487]:
#Base de topografia

base_topografia = pd.read_excel(path_topografia, sheet_name='SOP',skiprows=1, dtype={'Chave':'object', 'Chave Pai':'object', 'Data entrega SOP Topografia':'datetime64[ns]'}, usecols=['Data entrega SOP Topografia', 'Chave'])

base_topografia = mascara(base_topografia,'Chave',7)

base_topografia = base_topografia.drop_duplicates('Chave')


In [488]:
retalhonamentos_justificado = pd.read_excel(path_justificativas , sheet_name='retalhonamento', dtype={'objeto de locação origem':'object', 'objeto de locação retalhonado':'object'})

In [489]:
arquivos_retalhonamentos = []



# Caminhar pela árvore de diretórios
for root, dirs, files in os.walk(path_retalhonamento):
    for a in files:
        if a.endswith('.xlsx') and 'de_para' in a.lower() and '~' not in a and not 'parcial' in a.lower() :
            file_path = os.path.join(root, a)
            creation_time = os.path.getctime(file_path)
            creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d')
            origem = os.path.basename(root)
            nome_do_arquivo = a
            arquivos_retalhonamentos.append((file_path, a, creation_date, origem))
            


colunas_ret = ['Arquivo', 'nome do arquivo', 'data','diretorio']
links_retalhonamento = pd.DataFrame(arquivos_retalhonamentos, columns=colunas_ret)
links_retalhonamento = links_retalhonamento.sort_values(['data'], ascending=False).drop_duplicates(['Arquivo'], keep='first')



retalhonamentos = pd.DataFrame()
for _, n in links_retalhonamento.iterrows():
    base = pd.read_excel(n['Arquivo'],skiprows=1)
    base['nome_do_arquivo'] = n['nome do arquivo']
    retalhonamentos = pd.concat([retalhonamentos,base], ignore_index=True)

In [490]:
#Retalhonamento Justificado Inclusão e exclusão

retalhonamentos_justificado = retalhonamentos_justificado[retalhonamentos_justificado['status retalhonamento'] == 'Incluir']


retalhonamentos_justificado = mascara(retalhonamentos_justificado,'objeto de locação origem',7)

retalhonamentos_justificado = mascara(retalhonamentos_justificado,'objeto de locação retalhonado',7)

#Definindo ID_PROJETO
retalhonamentos_justificado['ID_PROJETO'] = retalhonamentos_justificado['objeto de locação origem'].str[:4]

#Definindo ID_PROJETO_1
retalhonamentos_justificado['ID_PROJETO_1'] = retalhonamentos_justificado['objeto de locação retalhonado'].str[:4]

#Definindo ID_TALHÃO
retalhonamentos_justificado['ID_TALHAO'] = retalhonamentos_justificado['objeto de locação origem'].str[-3:]

#Definindo ID_TALHÃO_1
retalhonamentos_justificado['ID_TALHAO_1'] = retalhonamentos_justificado['objeto de locação retalhonado'].str[-3:]

#Definindo as datas de retalhonamento, apenas copiando a primeira data

retalhonamentos_justificado['DATA_CRIAC'] = retalhonamentos_justificado['Data de retalhonamento']

retalhonamentos_justificado['AREA_HA'] = 9999


ret_rename = {
    'cd uso solo origem':'CD_USO_SOL',
    'cd uso solo retalhonado': 'CD_USO_SOL_1',
    'Data de retalhonamento': 'DATA_CRIAC_1'
    
    }
retalhonamentos_justificado = retalhonamentos_justificado.rename(ret_rename,axis=1)

#Alterar o tipo de dados para ficar igual a da base retalhonamentos e não ter problemas futuros no código

ret_jus_type = {'CD_USO_SOL':'float64',
                'CD_USO_SOL_1':'float64',
                'objeto de locação origem':'object',
                'objeto de locação retalhonado':'object',
                'DATA_CRIAC':'datetime64[ns]',
                'DATA_CRIAC_1':'datetime64[ns]'
                }

retalhonamentos_justificado = retalhonamentos_justificado.astype(ret_jus_type)



#Criar uma base para exclusão de retalhonamentos
#Essa base só precisa de cd_uso_origem para excluir a informação da base original

retalhonamentos_justificado_excluir = pd.read_excel(path_justificativas, sheet_name='retalhonamento', dtype={'cd uso solo origem':'int64'})

retalhonamentos_justificado_excluir = mascara(retalhonamentos_justificado_excluir,'objeto de locação origem',7)

retalhonamentos_justificado_excluir = retalhonamentos_justificado_excluir[retalhonamentos_justificado_excluir['status retalhonamento'] == 'Excluir']


In [491]:
#Escolher colunas para importar base de retalhonamento


retalhonamentos = retalhonamentos[['CD_USO_SOL','ID_PROJETO', 'ID_TALHAO','AREA_HA', 'DATA_CRIAC', 'CD_USO_SOL_1','ID_PROJETO_1', 'ID_TALHAO_1', 'DATA_CRIAC_1']]


#Criando coluna de apoio para retalhonamentos

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO',3)
retalhonamentos['objeto de locação origem'] = retalhonamentos['ID_PROJETO'] + retalhonamentos['ID_TALHAO']
retalhonamentos = mascara(retalhonamentos,'objeto de locação origem',7)

#Preenchendo campos vazios com zero para transformar em str
retalhonamentos['ID_PROJETO_1'] = retalhonamentos['ID_PROJETO_1'].fillna(0).astype(int).astype(str)
retalhonamentos['ID_TALHAO_1'] = retalhonamentos['ID_TALHAO_1'].fillna(0).astype(int).astype(str)

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO_1',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO_1',3)
retalhonamentos['objeto de locação retalhonado'] = retalhonamentos['ID_PROJETO_1'] + retalhonamentos['ID_TALHAO_1']
retalhonamentos = mascara(retalhonamentos,'objeto de locação retalhonado',7)


#Exportar o retalhonamento

retalhonamentos.to_excel('Retalhonamento Geo.xlsx')



#Inserir exclusão de apontamentos

retalhonamentos = retalhonamentos.merge(retalhonamentos_justificado_excluir[['cd uso solo origem', 'status retalhonamento']], left_on='CD_USO_SOL', right_on='cd uso solo origem', how='left').drop('cd uso solo origem',axis=1)

#Filtrando tudo que for diferente de excluir, garantindo que os retalhonamento excluídos não estejam na base
retalhonamentos = retalhonamentos[retalhonamentos['status retalhonamento'] != 'Excluir']

#Inserir retalhonamentos justificados

retalhonamentos = pd.concat([retalhonamentos,retalhonamentos_justificado])





# Alterando o tipo de dado para retalhonamento
retalhonamentos['CD_USO_SOL'] = retalhonamentos['CD_USO_SOL'].fillna(0).astype(int)
retalhonamentos['CD_USO_SOL_1'] = retalhonamentos['CD_USO_SOL_1'].fillna(0).astype(int)



# Definindo o tipo de dado
dtype_retalhonamento = {'DATA_CRIAC':'datetime64[ns]',
                         'DATA_CRIAC_1':'datetime64[ns]',
                         'CD_USO_SOL':'object',
                         'CD_USO_SOL_1':'object'}

retalhonamentos = retalhonamentos.astype(dtype_retalhonamento)


# Remover apontamentos anteriores a 2020

retalhonamentos = retalhonamentos[(retalhonamentos['DATA_CRIAC'].dt.year >= 2020) &  (retalhonamentos['DATA_CRIAC_1'].dt.year >= 2020)]


# Filtrando o que não tem cd_uso_solo definido
retalhonamentos = retalhonamentos[(pd.notna(retalhonamentos['CD_USO_SOL'])) | (pd.notna(retalhonamentos['CD_USO_SOL_1']) ) ]

# Encontrando o que temos de retalhonamento
# Precisamos que os talhões sejam diferentes tanto objeto de locação quanto o cd_uso_solo se qualquer um dos dois for, excluir
retalhonamentos['delta_cd_uso_sol'] = (retalhonamentos['CD_USO_SOL'] == retalhonamentos['CD_USO_SOL_1'])
retalhonamentos['delta_talhao'] = (retalhonamentos['objeto de locação origem'] == retalhonamentos['objeto de locação retalhonado'])


# Filtrando somente talhões que tem identificação de retalhonameto
retalhonamentos = retalhonamentos[retalhonamentos['delta_cd_uso_sol'] == False]
retalhonamentos = retalhonamentos[retalhonamentos['delta_talhao'] == False]


# Retirar talhões zero da base
retalhonamentos = retalhonamentos[retalhonamentos['CD_USO_SOL'] != 0]
retalhonamentos = retalhonamentos[retalhonamentos['CD_USO_SOL_1'] != 0]


# Remover talhões vindos de retalhonamentos, sempre mantendo o que tem a maior área
retalhonamentos = retalhonamentos.sort_values(['CD_USO_SOL_1', 'AREA_HA'], ascending=[False,False])
retalhonamentos = retalhonamentos.drop_duplicates('CD_USO_SOL_1', keep='first')


# Organizar talhões de forma ascendente
retalhonamentos = retalhonamentos.sort_values(['CD_USO_SOL','CD_USO_SOL_1', 'DATA_CRIAC_1'], ascending=[False,False, True])

retalhonamentos = retalhonamentos.rename({'CD_USO_SOL':'cd_uso_solo_origem', 
                                           'CD_USO_SOL_1':'cd_uso_solo_retalhonado',
                                           'DATA_CRIAC':'Data de criação origem',
                                           'DATA_CRIAC_1':'Data de criação retalhonado'
                                           },axis=1)


# Removendo onde temos duplicatas em todas as linhas
retalhonamentos = retalhonamentos.drop_duplicates(keep='first')


# Criar lógica para encontrar se temos talhões iguais, mas com datas distintas
retalhonamentos['erro cadastro'] = (
     (retalhonamentos['cd_uso_solo_origem'] == retalhonamentos['cd_uso_solo_origem'].shift(1)) &
         (retalhonamentos['cd_uso_solo_retalhonado'] == retalhonamentos['cd_uso_solo_retalhonado'].shift(1)) &
             (retalhonamentos['Data de criação retalhonado'] != retalhonamentos['Data de criação retalhonado'].shift(1))
  )

# Selecionando somente o que é igual, excluindo o que é True

retalhonamentos = retalhonamentos[retalhonamentos['erro cadastro'] != True ]


retalhonamentos.to_excel('retalhonamento ajustado.xlsx')


In [492]:
justificativas = pd.read_excel(path_justificativas,sheet_name='justificativas',engine='openpyxl')

In [493]:
sequencia_vigente = pd.read_excel(path_justificativas, sheet_name='Sequência de Operações', engine='openpyxl')

In [494]:
inclusao_boletim = pd.read_excel(path_justificativas,sheet_name='inclusão de boletim', engine='openpyxl')

In [495]:
#Base de material genético
#Base atualizada em premissas 

classe_mtg = pd.read_excel(path_premissas, sheet_name='material_genetico')
classe_mtg = classe_mtg.drop_duplicates('Material Genético', keep='first')
classe_mtg = classe_mtg.dropna(how= 'all')


In [496]:
#Base Cadastro Ciclo

cadastro_ciclo = mascara(cadastro_ciclo,'id_projeto',4)
cadastro_ciclo = mascara(cadastro_ciclo,'cd_talhao',3)
cadastro_ciclo['objeto de locação'] = cadastro_ciclo['id_projeto'].astype(str) + cadastro_ciclo['cd_talhao'].astype(str)
cadastro_ciclo['vlr_area'] = cadastro_ciclo['vlr_area'].astype(float)
#Filtrando regimes 5 (Implantação) e 6 (Reforma) 
cadastro_ciclo = cadastro_ciclo[(cadastro_ciclo['cd_regime'] == 5) | (cadastro_ciclo['cd_regime']==6)]

#Filtrando somente o que é talhão para crescimento cd_caracteristica_uso_solo -> Não é possível filtrar o status. Caso contrário a regra do ciclo não irá funcionar

#Definindo vlr entrelinha 
cadastro_ciclo.loc[:,'vlr_entrelinha'] = cadastro_ciclo['dcr_espacamento'].str.split(' X ').str[0].str.replace(',','.').astype('float')
#Definindo vlr linha
cadastro_ciclo.loc[:,'vlr_linha'] = cadastro_ciclo['dcr_espacamento'].str.split(' X ').str[1].str[0].str.replace(',','.').astype('float')

#Trazendo material genético para a base de cadastro ciclo
cadastro_ciclo = cadastro_ciclo.merge(material_genetico[['cd_material_genetico','dcr_material_genetico']], left_on='cd_material_genetico', right_on='cd_material_genetico', how='left')

#Trazendo classe de plantio realizado para cadastro_ciclo
cadastro_ciclo = cadastro_ciclo.merge(classe_mtg, left_on='dcr_material_genetico', right_on='Material Genético',how='left').drop('Material Genético',axis=1).rename({'Classe MG Plantio':'Classe de Material Genético Plantado'}, axis=1)



In [497]:
#Base CTO

#Convertendo para o formato de data
cto_baldeio['data_cto'] = pd.to_datetime(cto_baldeio['data_cto'])

#Ordenando de forma a manter o apontamento mais rescente primeiro
cto_baldeio = cto_baldeio.sort_values(['cd_uso_solo','data_cto'], ascending=[False,False])

#Revomendo duplicatas para ter apenas o apontamento mais rescente
cto_baldeio = cto_baldeio.drop_duplicates('cd_uso_solo',keep='first')


cto_baldeio = mascara(cto_baldeio,'id_projeto',4)
cto_baldeio = mascara(cto_baldeio,'cd_talhao',3)
cto_baldeio['objeto de locação'] = cto_baldeio['id_projeto'] + cto_baldeio['cd_talhao']



#Adicionar o pas 


cto_baldeio = cto_baldeio.rename({'data_cto':'data_cto_baldeio'},axis=1)


cto_baldeio = cto_baldeio[pd.notna(cto_baldeio['data_cto_baldeio'])]

#cto_baldeio.to_excel('cto_baldeio.xlsx')


#Importante voltar para definir se usaremos o objeto de locação ou se iremos utilizar o código de uso do solo

In [498]:
#Base PAS

pas = pd.read_excel(path_pas,skiprows=1, usecols=['Id Projeto','Talhão','Data liberação ajustada']) 

pas['Data liberação ajustada'] = pd.to_datetime(pas['Data liberação ajustada'], errors='coerce')

pas = mascara(pas,'Id Projeto',4)

pas = mascara(pas,'Talhão',3)

pas['objeto de locação'] = pas['Id Projeto'] + pas['Talhão']

pas = pas.drop_duplicates('objeto de locação',keep='first')

#Adicionando informação ao PAS

cto_baldeio= pas.merge(cto_baldeio[['data_cto_baldeio','objeto de locação','dcr_produto']],left_on='objeto de locação', right_on='objeto de locação', how='left')

cto_baldeio['data_cto_baldeio'] = np.where(pd.isna(cto_baldeio['data_cto_baldeio']), cto_baldeio['Data liberação ajustada'], cto_baldeio['data_cto_baldeio'])



In [499]:
#Adição do CTO

adicao_cto = pd.read_excel(path_justificativas,usecols=['objeto de locação','data de cto','tipo de cto'], sheet_name='inclusão cto', dtype={'data de cto': 'datetime64[ns]'})

adicao_cto = mascara(adicao_cto,'objeto de locação',7)

adicao_cto = adicao_cto.sort_values('data de cto', ascending=False)

adicao_cto = adicao_cto[adicao_cto['tipo de cto'] == 'Baldeio']


adicao_cto = adicao_cto.drop('objeto de locação',axis=1)

adicao_cto = adicao_cto.rename({'data de cto':'data_cto_baldeio', 'tipo de cto':'dcr_produto'},axis=1)

#Adição do CTO e PAS
cto_baldeio = pd.concat([adicao_cto,cto_baldeio])


In [500]:
def mes_operacional(df, coluna, nome_coluna):
    # Garantir que a coluna é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df[coluna]):
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    
    
    def calcular_mes_operacional(data):
        
        if pd.isna(data):
            return None 
        
        
        dia = data.day
        mes = data.month
        ano = data.year
        
        
        if dia > 20:
            mes = (mes % 12) + 1
            if mes == 1:
                ano += 1
        
        # Retornar um Timestamp, garantindo que todos são inteiros
        return pd.Timestamp(year=int(ano), month=int(mes), day=1)

    # Aplicar a função para a coluna
    df[nome_coluna] = df[coluna].apply(calcular_mes_operacional)

    return df
    

In [501]:
#Base de justificativas -> Talhões justificados para fitrarem o boletim

justificativas = mascara(justificativas,'Objeto de locação',7)


In [502]:
#Data de Plantio

data_de_plantio = cadastro_ciclo[['objeto de locação','data_plantio','cd_uso_solo','dcr_material_genetico','Classe de Material Genético Plantado']].rename({'dcr_material_genetico':'Material Genético Plantado'}, axis=1)

data_de_plantio = data_de_plantio.rename({'data_plantio':'Data de Plantio'},axis=1)

data_de_plantio = data_de_plantio[data_de_plantio ['Data de Plantio'].notna()]

data_de_plantio['Data de Plantio'] = pd.to_datetime(data_de_plantio['Data de Plantio'], dayfirst=True)

#Alterar o filtro para a data de plantio, precisa ser menor do que hoje
data_de_plantio = data_de_plantio[data_de_plantio['Data de Plantio'] < today]

In [503]:
#Bases de premissas
insumos = pd.read_excel(path_premissas, sheet_name='insumos',engine = 'openpyxl')

operacao = pd.read_excel(path_premissas, sheet_name='operacao',engine = 'openpyxl')

regua_operacao = pd.read_excel(path_premissas, sheet_name='regua_operacao',engine = 'openpyxl')

tipo_aprovacao = pd.read_excel(path_premissas, sheet_name='status')

In [504]:
#Trabalhando as recomendações de adubações


recomendacao_adu = pd.read_excel(path_recomendacao_adu,sheet_name='Enviados_Silvicultura')

colunas_rec_adu = {
    recomendacao_adu.columns[1]: 'Projeto',
    recomendacao_adu.columns[3]: 'Talhão',
    recomendacao_adu.columns[6]: 'Insumo mecanizado',
    recomendacao_adu.columns[7]: 'Dose kg/ha calagem',
    recomendacao_adu.columns[8]: 'Insumo manual',
    recomendacao_adu.columns[9]: 'Dose kg/ha calagem 2',
    recomendacao_adu.columns[10]: 'Insumo base',
    recomendacao_adu.columns[11]: 'Dose kg/ha base',
    recomendacao_adu.columns[12]: 'Insumo cobertura',
    recomendacao_adu.columns[13]: 'Dose kg/ha cobertura',
    recomendacao_adu.columns[15]: 'Insumo cobertura 2',
    recomendacao_adu.columns[16]: 'Dose kg/ha cobertura 2',
    recomendacao_adu.columns[22]: 'Data de recomendação'
}

#Renomeando as colunas
recomendacao_adu = recomendacao_adu.rename(columns=colunas_rec_adu)

#Escolhendo colunas
recomendacao_adu = recomendacao_adu[['Projeto','Talhão','Insumo mecanizado','Dose kg/ha calagem','Insumo manual','Dose kg/ha calagem 2','Insumo base','Dose kg/ha base','Insumo cobertura','Dose kg/ha cobertura','Insumo cobertura 2','Dose kg/ha cobertura 2','Data de recomendação']]

#Removendo a primeira linha
recomendacao_adu = recomendacao_adu.iloc[1:]

#Aplicando máscara
recomendacao_adu = mascara(recomendacao_adu,'Projeto',4)
recomendacao_adu = mascara(recomendacao_adu,'Talhão',3)

#Coluna objeto de locação
recomendacao_adu['Objeto de locação'] = recomendacao_adu.apply(lambda x: x['Projeto'] + x['Talhão'], axis=1)

#Alterando o tipo de data
recomendacao_adu['Data de recomendação'] = pd.to_datetime(recomendacao_adu['Data de recomendação'])

#ordenando o data frame
recomendacao_adu = recomendacao_adu.sort_values(['Objeto de locação','Data de recomendação'], ascending=[True,False])

#Mantendo apenas os itens mais recentes

recomendacao_adu = recomendacao_adu.drop_duplicates('Objeto de locação',keep='first')

#Trazendo somente o que preciso no merge, sendo operações de base e removendo duplicatas em formulação
recomendacao_adu = recomendacao_adu.merge(
    insumos[insumos['Tipo de Adubação'] == 'Base'].drop_duplicates('Formulação')[['Formulação','Estratégia']] ,      
    left_on='Insumo base', right_on='Formulação', how='left').drop('Formulação',axis=1)

recomendacao_adu['Recomendação de Calagem'] = recomendacao_adu.apply(lambda x: x['Insumo manual'] if pd.isna(x['Insumo mecanizado']) else x['Insumo mecanizado'], axis=1 )
recomendacao_adu['Recomendação Dose kg/ha Calagem'] = recomendacao_adu.apply(lambda x: x['Dose kg/ha calagem 2'] if pd.isna(x['Dose kg/ha calagem']) else x['Dose kg/ha calagem'], axis=1 )

recomendacao_adu = recomendacao_adu.rename({'Estratégia': 'Estratégia Recomendada'}, axis=1)

#Inciando o processo de recomendação complementar

recomendacao_comp =  pd.read_excel(path_rec_complementar, sheet_name='Enviados Silvicultura')

colunas_comp = {
recomendacao_comp.columns[2] : 'Projeto',
recomendacao_comp.columns[4] : 'Talhão',
recomendacao_comp.columns[10] : 'Status',
recomendacao_comp.columns[11] : 'NPK complementar',
recomendacao_comp.columns[12] : 'Dose kg/ha complementar',
recomendacao_comp.columns[13] : 'Corretivos complementares',
recomendacao_comp.columns[14] : 'Dose kg/ha corretivos',
recomendacao_comp.columns[16] : 'Data de recomendação'
}
recomendacao_comp = recomendacao_comp.rename(columns=colunas_comp)

#Escolhendo colunas
recomendacao_comp = recomendacao_comp[['Projeto','Talhão','Status','NPK complementar','Dose kg/ha complementar','Corretivos complementares','Dose kg/ha corretivos','Data de recomendação']]

#Excluindo a primeira linha
recomendacao_comp = recomendacao_comp.iloc[1:]

recomendacao_comp['Data de recomendação'] = pd.to_datetime(recomendacao_comp['Data de recomendação'])


recomendacao_comp = mascara(recomendacao_comp,'Projeto',4)
recomendacao_comp = mascara(recomendacao_comp,'Talhão',3)
recomendacao_comp['Objeto de locação'] = recomendacao_comp.apply(lambda x: x['Projeto'] + x['Talhão'],axis=1)

#Ordenando o data frame
recomendacao_comp = recomendacao_comp.sort_values(['Objeto de locação','Data de recomendação'], ascending=[True, False])

#Exluindo duplicatas e mantendo somente o primeiro
recomendacao_comp = recomendacao_comp.drop_duplicates('Objeto de locação',keep='first')

#Filtrando somente o que tem recomendação realizada
recomendacao_comp = recomendacao_comp[recomendacao_comp['Status'] == 'Recomendação realizada']

#o que importa é que no nome tenha a palavra oxyfertil
recomendacao_comp['Corretivos complementares'] = recomendacao_comp['Corretivos complementares'].apply(lambda x: 'Oxyfertil' if 'Oxyfertil' in x else x)

#Excluindo onde todas as colunas são iguais a n.a ou nulas
recomendacao_comp = recomendacao_comp[((recomendacao_comp['NPK complementar'] != 'n.a.') & (pd.notna(recomendacao_comp['NPK complementar']))) | ((recomendacao_comp['Corretivos complementares'] != 'n.a.') & (pd.notna(recomendacao_comp['Corretivos complementares'])))]


In [505]:
#Ler o Boletim

boletim1 = pd.read_parquet(df_boletim1, engine='fastparquet')

boletim2 = pd.read_parquet(df_boletim2, engine='fastparquet')

boletim3 = pd.read_parquet(df_boletim3, engine='fastparquet')

boletim4 = pd.read_parquet(df_boletim4, engine='fastparquet')

boletim5 = pd.read_parquet(df_boletim5, engine='fastparquet')



boletim = pd.concat([boletim1,boletim2,boletim3,boletim4, boletim5], ignore_index=True)


# Juntar todos os boletins em apenas um e arquivá-lo dentro do excel ou parquet, precisa ter uma coluna com o mês operacional de fechamento

# Se hoje é dia 24, então ele salva o arquivo e isola ele como boletim(mês:ano) operacional de silvicultura;
    #Flag de data Gabriela, se for definido deve sobrepor regra
    #Ativar flag e data de fechamento;
    #Se for executado e tiver um flag no excel para rodar somente com esse boletim(mes:ano), execute;
    #Salvar em excel;
    #Front end, o que executar e quando executar;
    #Todo o script deve ser executado com esse boletim de acordo com as premisas;
    #Criar uma função que verifica data e flags;
    # Return boletim;
    #calendar;
    #salvar boletim oficial    
     




#boletim = pd.concat([boletim3,boletim4, boletim5], ignore_index=True)

boletim = boletim[['ID_REGIAO','CD_BOLETIM_SILVICULTURA','DATA_OPERACAO','VLR_PRODUCAO','DCR_OPERACAO','CD_TALHAO','ID_PROJETO','CD_PROJETO','NOME_RECURSO','QUANTIDADE','UNIDADE_MED','DCR_FASE_PRESCRICAO','FLAG_OCORRENCIA','NOM_PROJETO','TIP_APROVACAO','CD_MATERIAL_GENETICO']]

#Filtrando apenas São Paulo - IDREGIÃO == 1 e 2

boletim = boletim[ (boletim['ID_REGIAO'] == 1) | (boletim['ID_REGIAO'] ==2 )]

#Filtrando talhões que não são nulos -> Operações que não são necessárias apontamento de talhão -> Ex: Caminhão Munck
boletim_talhao_null = boletim['CD_TALHAO'].notna()
boletim = boletim[boletim_talhao_null]

boletim = mascara(boletim,'ID_PROJETO',4)
boletim = mascara(boletim, 'CD_TALHAO',3)

boletim['objeto de locação'] = boletim['ID_PROJETO'].astype(str) + boletim['CD_TALHAO'].astype(str)

In [506]:
#Inclusão de boletim 

#Mesma base onde temos justificativas;
#os boletins são genéricos
#se houverem boletins duplicados devido a utilização de mais de um insumo, iremos deixá-lo cair nas regras, o próprio código vai definir qual deverá ser utilizado. Não é necessário remover


#Garantir que o objeto de locação tenha 7 digitos
inclusao_boletim = mascara(inclusao_boletim,'objeto de locação',7)

#Renomear as colunas para manter o mesmo padrão do boletim 
inclusao_boletim = inclusao_boletim.rename({
    'data de operação' : 'DATA_OPERACAO',
    'operação' : 'DCR_OPERACAO',
    'insumo': 'NOME_RECURSO',
    'produção (ha)' : 'VLR_PRODUCAO',
    'quantidade' : 'QUANTIDADE',
    'boletim': 'CD_BOLETIM_SILVICULTURA'
}, axis=1)



#Alterando o tipo de dados para realizar a concatenação

types_inclusao = {
    'CD_BOLETIM_SILVICULTURA': 'float64',
    'objeto de locação': 'object',
    'DATA_OPERACAO': 'datetime64[us]',
    'DCR_OPERACAO':'object',
    'VLR_PRODUCAO': 'float64',
    'QUANTIDADE': 'float64',
    'motivo de inclusão' : 'object'    
}

inclusao_boletim = inclusao_boletim.astype(types_inclusao)


#Criar coluna para definir que os boletins estão aprovados

#Definindo a aprovação dos boletins
inclusao_boletim['TIP_APROVACAO'] = 'A'

#Concatenando boletim com a inclusão de novos boletins
boletim = pd.concat([boletim, inclusao_boletim], ignore_index=True)



In [507]:
#Cadastro e Ciclo

#Trazendo o ciclo do cadastro para o boletim

boletim = boletim.merge(cadastro_ciclo[['objeto de locação','nom_projeto','num_ciclo','num_rotacao','data_reg','cd_regime','cd_uso_solo','vlr_area','vlr_entrelinha','vlr_linha','Classe de Material Genético Plantado']], left_on='objeto de locação', right_on='objeto de locação', how='left')

#Filtrando apenas o que é regime de implantação e reforma
boletim = boletim[(boletim['cd_regime'] == 5) | (boletim['cd_regime'] == 6) ]


delta_ciclo_max = 0

delta_ciclo_min = -60


boletim['delta_teste'] = (boletim['DATA_OPERACAO'] - boletim['data_reg']).dt.days



#Definindo o que é ciclo vigente
boletim['ciclo_vigente_teste'] = boletim.apply(lambda x: 'Ciclo Vigente' if (x['DATA_OPERACAO'] >= x['data_reg']) or (((x['DATA_OPERACAO']-x['data_reg']).days <= delta_ciclo_max) and  ((x['DATA_OPERACAO']-x['data_reg']).days >= delta_ciclo_min)) else 'Não Vigente',axis=1)

#Removendo os ciclos que não são vigentes


boletim['ciclo_vigente'] = boletim.apply(lambda x: 'Ciclo Vigente' if (x['DATA_OPERACAO'] >= x['data_reg']) else 'Não Vigente',axis=1)

#boletim['check'] = boletim['ciclo_vigente_teste'] == boletim['ciclo_vigente']
#boletim = boletim[boletim['ciclo_vigente'] == 'Ciclo Vigente']


boletim = boletim[boletim['ciclo_vigente_teste'] == 'Ciclo Vigente']


#Ordenando o boletim
boletim = boletim.sort_values(['objeto de locação','CD_BOLETIM_SILVICULTURA','data_reg'],ascending=[False,False,False])

#Removendo duplicatas para ter somente um único boletim
boletim = boletim.drop_duplicates(['objeto de locação','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'],keep='first')

#Removendo cd_uso_solo vazios - Quando é removido o 'cd_uso_solo' vazio, garante que teremos apenas 
boletim = boletim[boletim['cd_uso_solo'].notna()]


In [508]:
#Trazendo o retalhonamento para a base de boletim
#Trazendo apenas uma cópia do boletim, não é necessário utilizá-lo e perder suas referências duplicando ele
boletim_retalhonamento = boletim.merge(retalhonamentos[['cd_uso_solo_origem','cd_uso_solo_retalhonado','Data de criação retalhonado','ID_PROJETO_1','ID_TALHAO_1','objeto de locação retalhonado']], left_on='cd_uso_solo', right_on='cd_uso_solo_origem', how='left')


In [509]:
#Criar regra para saber se o retalhonamento aconteceu antes ou depois da operação

#Tratando os boletins retalhonados

#Se teve data de operação antes de ser retalhonado, vida que segue, caso contrário
print(f"boletim inicio {boletim_retalhonamento.shape[0]}")

#Encontrando o que é retalhonamento após a operação
boletim_retalhonamento['status retalhonamento'] =  boletim_retalhonamento.apply(lambda x: 'Retalhonado após operação' if  ((x['Data de criação retalhonado'] - x['DATA_OPERACAO']).days) >= 0 else 'Não havia retalhonamento', axis=1)

#Em que momento retirei os talhões que não tinham apontamento após?
#Não precisei remover se estou utilizando um df intermediário

#Filtrando somente o que é retalhonamento após operação
boletim_retalhonamento = boletim_retalhonamento[boletim_retalhonamento['status retalhonamento'] == 'Retalhonado após operação']


#Identificando quais são os retalhonamentos para seguir as regras
boletim_retalhonamento['retalhonamento_historico'] = 1

#Valores históricos, informações de retalhonamento
print(f'boletins com histórico de retalhonametentos { boletim_retalhonamento.shape[0]}')

#Reorganizar os boletins, somá-los a 10MM. Caso contrário podemos ter duplicatas e o código considerar apenas um único boletim
#Todos os boletins terão index de 10MM
boletim_retalhonamento['CD_BOLETIM_SILVICULTURA'] = boletim_retalhonamento['CD_BOLETIM_SILVICULTURA']+ 10000000

#Definir quais são os talhões retalhonados após a operação e que tenham o retalhonamento_historico == 1
    #Esses talhões deverão ter o código de uso de solo atribuído como o novo cd_uso_solo

boletim_retalhonamento['cd_uso_solo'] = np.where( (boletim_retalhonamento['retalhonamento_historico'] == 1) & (boletim_retalhonamento['status retalhonamento'] == 'Retalhonado após operação'),
                                                boletim_retalhonamento['cd_uso_solo_retalhonado'], boletim_retalhonamento['cd_uso_solo'])

boletim_retalhonamento['CD_TALHAO'] = np.where( (boletim_retalhonamento['retalhonamento_historico'] == 1) & (boletim_retalhonamento['status retalhonamento'] == 'Retalhonado após operação'),
                                                boletim_retalhonamento['ID_TALHAO_1'], boletim_retalhonamento['CD_TALHAO'])

boletim_retalhonamento['ID_PROJETO'] = np.where( (boletim_retalhonamento['retalhonamento_historico'] == 1) & (boletim_retalhonamento['status retalhonamento'] == 'Retalhonado após operação'),
                                                boletim_retalhonamento['ID_PROJETO_1'], boletim_retalhonamento['ID_PROJETO'])

boletim_retalhonamento['objeto de locação'] = np.where( (boletim_retalhonamento['retalhonamento_historico'] == 1) & (boletim_retalhonamento['status retalhonamento'] == 'Retalhonado após operação'),
                                                boletim_retalhonamento['objeto de locação retalhonado'], boletim_retalhonamento['objeto de locação'])




   
        #O número de linhas retalhonadas sempre deverá ser grande, pois ele pega todo o histórico do talhão e atribuí como o talhão retalhonado

            #posso deixar o número no script apenas para identificar como ficou o histórico

                #Boletim + 1mm, caso contrário nossas regras vão remover os boletins

#Após definir como novo talhão, posso executar o código e entender o que houve com os talhões que tem replantio apontado. Pois eles já cairam diretamente na lógica.



#Não esquecer de levar o cd_uso_solo_origem para a base de aderência, para saber qual era o talhão retalhonado
boletim_retalhonamento['duplicatas'] = boletim_retalhonamento.duplicated(keep=False)
boletim_retalhonamento = boletim_retalhonamento.drop_duplicates(keep='first')


print(f"boletins com retalhonamento {boletim_retalhonamento.shape[0]}")




boletim inicio 386068
boletins com histórico de retalhonametentos 29684
boletins com retalhonamento 29684


In [510]:
boletim_retalhonamento

,ID_REGIAO,CD_BOLETIM_SILVICULTURA,DATA_OPERACAO,VLR_PRODUCAO,DCR_OPERACAO,CD_TALHAO,ID_PROJETO,CD_PROJETO,NOME_RECURSO,QUANTIDADE,...,ciclo_vigente,cd_uso_solo_origem,cd_uso_solo_retalhonado,Data de criação retalhonado,ID_PROJETO_1,ID_TALHAO_1,objeto de locação retalhonado,status retalhonamento,retalhonamento_historico,duplicatas
65,1.0,10458287.0,2024-03-18,0.18,SERV ROCADA MANUAL N3 AGRIC,004,2411,1900.0,None,NaN,...,Ciclo Vigente,101751,98251,2024-06-11,2411,004,2411004,Retalhonado após operação,1,False
79,1.0,10492396.0,2024-05-29,3.00,Savannah Rebaixamento/Subsolagem Propria,003,2411,1900.0,[24069] Tratores Savanah,2.8,...,Ciclo Vigente,102110,98250,2024-06-11,2411,003,2411003,Retalhonado após operação,1,False
80,1.0,10492396.0,2024-05-29,3.00,Savannah Rebaixamento/Subsolagem Propria,003,2411,1900.0,[27006] Tratores Savanah,3.7,...,Ciclo Vigente,102110,98250,2024-06-11,2411,003,2411003,Retalhonado após operação,1,False
81,1.0,10491820.0,2024-05-31,2.80,Adub. Base e Capina Savanah Propria,003,2411,1900.0,[20022] Subsolagem Norte,2.7,...,Ciclo Vigente,102110,98250,2024-06-11,2411,003,2411003,Retalhonado após operação,1,False
82,1.0,10491820.0,2024-05-31,2.80,Adub. Base e Capina Savanah Propria,003,2411,1900.0,[20024] Subsolagem Norte,2.3,...,Ciclo Vigente,102110,98250,2024-06-11,2411,003,2411003,Retalhonado após operação,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385746,1.0,10128229.0,2021-07-16,42.31,SERV COMB FORMIGA MEC 5 RUAS AGRIC,080,0001,1.0,FORMICIDA ISCA ATTA MEX-S 5KG,130.0,...,Ciclo Vigente,11141,103063,2024-04-15,0001,080,0001080,Retalhonado após operação,1,False
385747,1.0,10102243.0,2021-03-24,42.31,SERV COMB FORMIGA MANUAL 1 RUA AGRIC,080,0001,1.0,FORMICIDA ISCA ATTA MEX-S 5KG,350.0,...,Ciclo Vigente,11141,103063,2024-04-15,0001,080,0001080,Retalhonado após operação,1,False
385748,1.0,10057010.0,2020-09-15,42.31,SERV COMB FORMIGA MEC 5 RUAS AGRIC,080,0001,1.0,FORMICIDA ISCA ATTA MEX-S 5KG,158.0,...,Ciclo Vigente,11141,103063,2024-04-15,0001,080,0001080,Retalhonado após operação,1,False
385749,1.0,10020923.0,2020-05-02,42.31,SERV CAP QUIM MEC AREA TOTAL AGRIC,080,0001,1.0,HERBICIDA POS EMERGENTE XEQUE MATE 20LT,153.0,...,Ciclo Vigente,11141,103063,2024-04-15,0001,080,0001080,Retalhonado após operação,1,False


In [511]:
#Concatenando o boletim_retalhonado com o boletim

print(f'Número de boletins no boletim {boletim.shape[0]}')

boletim = pd.concat([boletim, boletim_retalhonamento])

print(f"boletins + boletins de retalhonamento {boletim.shape[0]}")


Número de boletins no boletim 376794
boletins + boletins de retalhonamento 406478


In [512]:
#Retirando os boletins que tem justificativas

boletim = boletim.merge(justificativas[['boletim','Boletim vigente justificativas']],left_on='CD_BOLETIM_SILVICULTURA', right_on='boletim', how='left' ).drop('boletim',axis=1)

boletim = boletim[boletim['Boletim vigente justificativas'] != "Não"]

In [513]:
#Encontrando operações e insumos que não estão cadastrados em premissas

#Delta de operações e insumo


operacao_boletim = set(boletim['DCR_OPERACAO'])
insumos_boletim = set(boletim['NOME_RECURSO'])

operacao_cadastro = set(operacao['Operações'])
insumos_cadastro = set(insumos['Insumo'])


delta_operacao= operacao_boletim - operacao_cadastro
delta_operacao.discard(None)

delta_insumo = insumos_boletim - insumos_cadastro 
delta_insumo.discard(None)

In [514]:
#Realizando merge para trazer as operações

boletim = boletim.merge(operacao[['Operações','Grupo de Operação','Operação Vigente','Tipo de operação']], left_on='DCR_OPERACAO', right_on='Operações', how='left').drop('Operações',axis=1)

#Filtrando DataFrame para remover todas as linhas nulas e manter apenas as linhas que tem operações cadastradas em premissas
linhas_nulas = boletim['Grupo de Operação'].notna()
boletim = boletim[linhas_nulas]


#Realizando merge para trazer os insumos

boletim = boletim.merge(insumos[['Insumo','Característica','Insumo Vigente','Fordor','Tipo de Adubação','Estratégia','Formulação']], left_on='NOME_RECURSO', right_on='Insumo', how='left').drop('Insumo',axis=1)

#Realizando merge para trazer as operações já conhecidas

boletim = boletim.merge(regua_operacao, left_on='DCR_OPERACAO', right_on='operação', how='left').drop('operação',axis=1)


#Status do boletim
boletim = boletim.merge(tipo_aprovacao[['Status','sigla','Status Vigente']], left_on='TIP_APROVACAO', right_on= 'sigla', how='left').drop('sigla',axis=1)

#Filtrando o status de acordo com o que está vigente em premissas

boletim = boletim[boletim['Status Vigente'] == 'Sim']


#Crianddo ID para encontrar a operação de plantio
boletim['id_plantio'] = boletim.apply(lambda x: 'Plantio' if x['Grupo de Operação'] == "Plantio" and x['Característica'] == 'Mudas' else None, axis=1)

In [515]:
#Boletim Topografia e Micro
boletim = boletim.merge(base_micro, left_on='objeto de locação', right_on=' INDEX', how='left').drop(' INDEX', axis=1)

boletim = boletim.merge(base_topografia, left_on='objeto de locação', right_on='Chave', how='left').drop('Chave', axis=1)



In [516]:
# Percentual de área apontada
boletim["%realizado"] = boletim.apply(lambda x: x['VLR_PRODUCAO']/ x['vlr_area'] if pd.notna(x['vlr_area']) else x['VLR_PRODUCAO'],axis=1)

#Removendo talhões com operações menores que 20% a não ser que ela seja replantio
boletim = boletim[(boletim['%realizado'] <= 0.20) & (boletim['Grupo de Operação'] == 'Replantio') |
                  (boletim['%realizado'] >= 0.20)
                  ]


#Filtrar operações e insumos vigentes
boletim = boletim[boletim['Operação Vigente'] == 'Sim']


#Removendo o filtro de Insumo vigente para todo o boletim, pode existir momentos em que a operação não tenha insumo
#boletim = boletim[boletim['Insumo Vigente'] == 'Sim']

#Ordenar por objeto de locação e por data (Ordem Decrescente)
boletim = boletim.sort_values(['objeto de locação','DATA_OPERACAO'], ascending=[True, False]) 

#Criar regra para remover duplicatas de boletins de plantio (Plantio com dois insumos -> Mudas Próprias e de terceiros)
duplicata_platio = (boletim['Grupo de Operação'] == 'Plantio')
#remover as duplicatas de acordo com a regra definida 
boletim.loc[duplicata_platio,:] = boletim.loc[duplicata_platio,:].drop_duplicates('CD_BOLETIM_SILVICULTURA')


In [517]:
#Código de plantio agrupado pelo ciclo

cod_plantio = boletim[boletim['Grupo de Operação'] == 'Plantio']
cod_plantio = cod_plantio.groupby('cd_uso_solo')['DATA_OPERACAO'].max().reset_index()

cod_plantio['Grupo de Operação'] = 'Plantio'
cod_plantio['cod_plantio'] = cod_plantio['cd_uso_solo'].astype(str) + cod_plantio['Grupo de Operação'].astype(str)

#Removendo talhões nulos
cod_plantio = cod_plantio[cod_plantio['cd_uso_solo'].notna()]



In [518]:
#Código de preparo agrupado pelo ciclo

cod_preparo = boletim[boletim['Grupo de Operação'] == 'Preparo']
cod_preparo = cod_preparo.groupby('cd_uso_solo')['DATA_OPERACAO'].max().reset_index()

cod_preparo['Grupo de Operação'] = 'Preparo'
cod_preparo['cod_preparo'] = cod_preparo['cd_uso_solo'].astype(str) + cod_preparo['Grupo de Operação']

In [519]:
#Realizando merge para trazer o campo cod_plantio

boletim = boletim.merge(cod_plantio, right_on=['cd_uso_solo', 'DATA_OPERACAO','Grupo de Operação'], left_on=['cd_uso_solo', 'DATA_OPERACAO','Grupo de Operação'], how= 'left')

boletim = boletim[boletim['cd_uso_solo'].notna()]

boletim = boletim.merge(cod_preparo, right_on=['cd_uso_solo','DATA_OPERACAO','Grupo de Operação'], left_on=['cd_uso_solo','DATA_OPERACAO','Grupo de Operação'], how='left')

boletim = boletim[boletim['cd_uso_solo'].notna()]

#Data de plantio de acordo com o cadastro
boletim = boletim.merge(data_de_plantio[['cd_uso_solo','Data de Plantio']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

#Data  de preparo de acordo com o cod_preparo
boletim['Data Preparo referência pre'] = boletim.apply(lambda x: x['DATA_OPERACAO'] if pd.notna(x['cod_preparo']) else None, axis=1)

In [520]:
#Remover boletim de retalhonamento se apontado após 60 dias de plantio

delta_plantio_retalhonamento_max = 60
delta_plantio_retalhonamento_min = -365


boletim.loc[boletim['status retalhonamento'] == 'Retalhonado após operação', 'delta_plantio_ret'] = \
    boletim.loc[boletim['status retalhonamento'] == 'Retalhonado após operação'].apply(
        lambda x: 0 if pd.isna(x['Data de Plantio']) else (x['DATA_OPERACAO'] - x['Data de Plantio']).days, axis=1
    )

boletim['delta_plantio_ret'] = boletim['delta_plantio_ret'].astype('float64')


#Se status retalhonamento for igual a Retalhonamento após operação e maior que delta_plantio_retalhonamento então ele deve ser retirado da base

boletim = boletim[((boletim['delta_plantio_ret'] >= delta_plantio_retalhonamento_min)  & (boletim['delta_plantio_ret'] <= delta_plantio_retalhonamento_max)) | (pd.isna(boletim['delta_plantio_ret'])) ] 

In [521]:
#Preparo de solos


#Quantidade maior que zero ou nula, devido aos apontamentos de operações manuais sem insumos
preparo_solo = boletim[(boletim['QUANTIDADE'] > 0) | (pd.isna(boletim['QUANTIDADE']))]

#Operação deve ser preparo e vigente
preparo_solo = preparo_solo[(preparo_solo['Grupo de Operação'] == 'Preparo') & (preparo_solo['Operação Vigente'] == 'Sim')]

#O insumo deve ter características de adubo e ser vigente ou a caracteristica ser nula devido a operações manuais sem apontamentos de insumos
preparo_solo = preparo_solo[(preparo_solo['Característica'] == 'Adubo') & (preparo_solo['Insumo Vigente'] != 'Não') | (pd.isna(preparo_solo['Característica']))]


#Encontrar deltra de preparo de solos em relação ao plantio

delta_preparo_plantio = 60

preparo_solo['delta_preparo_plantio'] = (preparo_solo['DATA_OPERACAO'] - preparo_solo['Data de Plantio']).dt.days

preparo_solo = preparo_solo[preparo_solo['delta_preparo_plantio'] <= delta_preparo_plantio]

#ordenando a tabela do maior para o menor
preparo_solo = preparo_solo.sort_values(['DATA_OPERACAO','CD_BOLETIM_SILVICULTURA','QUANTIDADE'], ascending=[False,False,False])

#Cada talhão deve ter apenas um único apontamento de preparo, por isso a necessidade de remover as duplicatas
preparo_solo = preparo_solo.drop_duplicates(subset=['cd_uso_solo'], keep='first')


preparo_solo['regra_de_operacao_preparo_solo'] = 'Preparo de Solos'


preparo_solo['Boletim Preparo de Solos'] = preparo_solo['CD_BOLETIM_SILVICULTURA']


preparo_solo = preparo_solo.rename({'DATA_OPERACAO':'Data Preparo de Solos'},axis=1)


#preparo_solo.to_excel('preparo_de_solo.xlsx')




In [522]:
#Savannah


savannah = boletim[boletim['QUANTIDADE'] > 0]

#Filtrando apenas operações de Savanna
savannah = savannah[savannah['Grupo de Operação'] == 'Savannah']

#Ordenando a tabela do maior para o menor
savannah = savannah.sort_values(['DATA_OPERACAO','CD_BOLETIM_SILVICULTURA'], ascending= [False,False])

#Removendo cd_uso_solo duplicados pois o talhão deve ter apenas um único boletim apontado de Savannah
savannah = savannah.drop_duplicates(subset=['cd_uso_solo'], keep= 'first')


savannah['regra_de_operacao_savannah'] = 'Savannah'


savannah['Boletim Savannah'] = savannah['CD_BOLETIM_SILVICULTURA']


savannah = savannah.rename({'DATA_OPERACAO':'Data Savannah'}, axis=1)


#savannah.to_excel('savannah.xlsx')


In [523]:
#Dessecação 

desseca = boletim[boletim['QUANTIDADE'] > 0]


# O grupo de operações precisa ser igual a herbicida e o insumo deve ser vigente
desseca = desseca[(desseca['Grupo de Operação'] == 'Herbicida') &  (desseca['Insumo Vigente'] == 'Sim')]

#Informação de CTO
desseca = desseca.merge(cto_baldeio[['objeto de locação', 'data_cto_baldeio']], left_on='objeto de locação' , right_on='objeto de locação', how='left')


#Informação de data de preparo de solos
#Realizar merge

desseca = desseca.merge(preparo_solo.drop_duplicates('cd_uso_solo')[['Data Preparo de Solos','cd_uso_solo']], left_on= 'cd_uso_solo', right_on = 'cd_uso_solo', how='left')

#Status de desseca (Depois do CTO e antes do Plantio)
desseca['Status desseca'] = desseca.apply(lambda x: 'Depois do CTO e antes do Plantio' if (x['DATA_OPERACAO'] >= x['data_cto_baldeio']) & (x['DATA_OPERACAO'] <= x['Data Preparo de Solos']) else 'Fora das premissas' , axis=1)

#O status deve ser igual a "Depois do CTO e antes do Plantio"
desseca = desseca[desseca['Status desseca'] == 'Depois do CTO e antes do Plantio']

#Ordenando o data Frame
desseca = desseca.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[False, False, False])


#Dose/ha
desseca['Dose/ha desseca'] = desseca['QUANTIDADE'] / desseca['VLR_PRODUCAO']

#Dose/ha acumulada
desseca['Dose/ha desseca acumulada'] = desseca.groupby('cd_uso_solo')['Dose/ha desseca'].transform('sum')

#Removendo talhões duplicados devido a varios insumos apontados
desseca = desseca.drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')


desseca['regra de mudança'] = (desseca['CD_BOLETIM_SILVICULTURA'] != desseca['CD_BOLETIM_SILVICULTURA'].shift(1)) & (desseca['cd_uso_solo'] != desseca['cd_uso_solo'].shift(1)) 

desseca['grupos distintos'] = desseca['regra de mudança'].cumsum()

desseca['Sequência de operação'] = desseca.groupby('grupos distintos').cumcount()+1

desseca['regra_de_operacao_desseca'] = np.where( desseca['Sequência de operação'] == 1, '1º Dessecação', 
                                                np.where( desseca['Sequência de operação'] == 2, '2º Dessecação', 
                                                         desseca['Sequência de operação'].map(lambda x: f'{x:.0f}').astype(str) + 'º Dessecação'
                                                        )
                                                )

desseca = desseca[desseca['Sequência de operação'] <=2]


desseca['Boletim Dessecação'] = desseca['CD_BOLETIM_SILVICULTURA']



desseca = desseca.rename({'DATA_OPERACAO':'Data de Dessecação', 'Sequência de operação' : 'Dessecação'}, axis=1)




In [524]:
#Formiga Pre Plantio

#transformando o boletim em uma base exclusiva de formiga pre plantio
formiga_pre_plant = boletim[boletim['QUANTIDADE'] > 0]

#Operação vigente
formiga_pre_plant = formiga_pre_plant[(formiga_pre_plant['Grupo de Operação'] =='Formiga Pre-plant') & (formiga_pre_plant['Operação Vigente'] == 'Sim')]

#Insumo Vigente
formiga_pre_plant = formiga_pre_plant[formiga_pre_plant['Insumo Vigente'] == 'Sim']

#Trazendo a data de CTO
formiga_pre_plant = formiga_pre_plant.merge(cto_baldeio[['data_cto_baldeio','dcr_produto','objeto de locação']], left_on='objeto de locação', right_on='objeto de locação', how='left').rename({'dcr_produto':'dcr_produto_cto'},axis=1)

#Status da operação
formiga_pre_plant['status'] = formiga_pre_plant.apply(lambda x: 'depois do CTO e antes do Plantio' if (x['DATA_OPERACAO'] >= x['data_cto_baldeio']) and (x['DATA_OPERACAO'] <= x['Data de Plantio']) else 'fora dos parametros' ,axis=1 )

#Filtrando talhões que apenas atendem ao critério
formiga_pre_plant = formiga_pre_plant[formiga_pre_plant['status'] == 'depois do CTO e antes do Plantio']

#Ordenando o data frame
formiga_pre_plant = formiga_pre_plant.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[False,False,False])



#antes de remover o insumo, preciso fazer uma soma acumulada deles
#Dose por ha
formiga_pre_plant['Dose/ha form pre plantio'] = formiga_pre_plant['QUANTIDADE'] / formiga_pre_plant['VLR_PRODUCAO']

#Dose acumulada
formiga_pre_plant['Dose acumulada/ha form pre plantio'] = formiga_pre_plant.groupby(['cd_uso_solo'])['Dose/ha form pre plantio'].transform('sum')

#Removendo duplicatas devido a  operações com diferentes insumos 
formiga_pre_plant = formiga_pre_plant.drop_duplicates(subset=['CD_BOLETIM_SILVICULTURA'], keep= 'first')


#Definindo a regra de operação 
formiga_pre_plant['regra de mudança'] = ((formiga_pre_plant['CD_BOLETIM_SILVICULTURA'] != formiga_pre_plant['CD_BOLETIM_SILVICULTURA'].shift(1)) & (formiga_pre_plant['cd_uso_solo'] != formiga_pre_plant['cd_uso_solo'].shift(1)))

formiga_pre_plant['grupos distintos'] = formiga_pre_plant['regra de mudança'].cumsum()

formiga_pre_plant['Sequência de operação'] = formiga_pre_plant.groupby('grupos distintos').cumcount()+1


#Regra de operação de formiga pre plantio

formiga_pre_plant['regra_operacao_form_pre_plantio'] = np.where(formiga_pre_plant['Sequência de operação'] == 1, '1º Formiga Pre Plantio'
                                                                , np.where( formiga_pre_plant['Sequência de operação'] == 2 ,'2º Formiga Pre Plantio',
                                                                            formiga_pre_plant['Sequência de operação'].map(lambda x: f'{x:.0f}').astype(str) + 'º Formiga Pre Plantio'
                                                                            
                                                                            )                                                                                                                             
                                                                )

#Filtrando apenas até a segunda operação

formiga_pre_plant = formiga_pre_plant[formiga_pre_plant['Sequência de operação'] <= 2]


#Boletim de formiga de pre plantio
formiga_pre_plant['boletim formiga pre plantio'] = formiga_pre_plant['CD_BOLETIM_SILVICULTURA']


formiga_pre_plant = formiga_pre_plant.rename({'DATA_OPERACAO':'Data de formiga pre plantio', 'Sequência de operação' : 'Formiga Pre Plantio'}, axis=1)


#formiga_pre_plant.to_excel('formiga pre plantio.xlsx')

In [525]:
#Replantio


#transformando o boletim em uma base exclusiva de replantio
replantio = boletim[boletim['QUANTIDADE'] > 0]

replantio = replantio[(replantio['Grupo de Operação'] == 'Replantio') & (replantio['Operação Vigente'] == 'Sim')]

replantio = replantio[(replantio['Característica'] == 'Mudas') & (replantio['Insumo Vigente'] == 'Sim')]

#Ordenando o data frame
replantio = replantio.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[False,True,False])
#Removendo as duplicatas para definir a ordem de operação
replantio_sduplicatas = replantio.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA',keep='first')

#Ordenando a base de sequência de operação para garantir que todos ficaram na mesma ordem

replantio_sduplicatas = replantio_sduplicatas.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[False,True])

replantio_sduplicatas['regra de mudança'] = (replantio_sduplicatas['CD_BOLETIM_SILVICULTURA'] != replantio_sduplicatas['CD_BOLETIM_SILVICULTURA'].shift(1)) & (replantio_sduplicatas['cd_uso_solo'] != replantio_sduplicatas['cd_uso_solo'].shift(1))

replantio_sduplicatas['grupos distintos'] = replantio_sduplicatas['regra de mudança'].cumsum()

replantio_sduplicatas['Sequência de operação'] = replantio_sduplicatas.groupby('grupos distintos').cumcount()+1


#levar a sequencia de operação de replantio para a base principal
replantio = replantio.merge(replantio_sduplicatas[['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência de operação']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'],how='left')

#Trazendo informação de código de material genético
replantio = replantio.merge(material_genetico[['cd_material_genetico','dcr_material_genetico','abrev_material_genetico']] , left_on=['CD_MATERIAL_GENETICO'], right_on=['cd_material_genetico'], how='left')

#Classe de material genético
replantio = replantio.merge(classe_mtg, left_on='dcr_material_genetico', right_on='Material Genético', how='left')





#Definindo o delta de plantio
replantio['Delta de plantio'] = replantio.apply(lambda x: (x['DATA_OPERACAO'] - x['Data de Plantio']).days if pd.notna(x['Data de Plantio']) else None, axis=1)


#Filtrando apontamentos que tenham replantio somente após o plantio
replantio = replantio[replantio['Delta de plantio'] > 0 ]


#Realizar o cálculo de mudas/ha e arredondar para cima
replantio['mudas/ha'] = np.ceil((replantio['QUANTIDADE'] / replantio['VLR_PRODUCAO']))

#mudas por m³
replantio['m3'] = replantio['vlr_entrelinha'] * replantio['vlr_linha']

#mudas/ha recomendado
replantio['mudas/ha recomendado'] = np.ceil((10000 / replantio['m3']))

#Replantio efetivo (total de mudas / mudas/ha recomendado)
replantio['Replantio Efetivo'] = round(replantio['mudas/ha'] / replantio['mudas/ha recomendado'],3)


#Criar a regra para definir qual é a primeira e a segunda operação
replantio['regra_operacao_replantio'] = np.where( replantio['Sequência de operação'] == 1 , '1º Replantio',
                                                 np.where( replantio['Sequência de operação'] == 2, '2º Replantio',
                                                        np.where( replantio['Sequência de operação'] == 3, '3º Replantio',                                              
                                                  replantio['Sequência de operação'].map(lambda x: f'{x:.0f}').astype(str) +'º Replantio'
                                                                )
                                                         )
                                                )

#Número de replantios no talhão
replantio['numero de replantios'] = replantio.groupby('cd_uso_solo')['Sequência de operação'].transform('max')


#Adicionar o export (Exportar todas as operações)
#replantio.to_excel('Replantio.xlsx')

#Filtrar somente até segunda operação

replantio = replantio[(replantio['regra_operacao_replantio'] == '1º Replantio') | (replantio['regra_operacao_replantio'] == '2º Replantio') | (replantio['regra_operacao_replantio'] == '3º Replantio') ]

#Criando referência do boletim
replantio['Boletim Replantio'] = replantio['CD_BOLETIM_SILVICULTURA']

replantio = replantio.rename({'DATA_OPERACAO':'Data Replantio', 'Sequência de operação':'Replantio'},axis=1)


In [526]:
mtg_vazio = replantio[['Boletim Replantio','CD_MATERIAL_GENETICO','cd_material_genetico','dcr_material_genetico','Material Genético', 'Classe MG Plantio']]

#set de materiais genéticos não cadastrados em premissas
sem_cadastro_mg = set(mtg_vazio[mtg_vazio['dcr_material_genetico'].notna() & mtg_vazio['Material Genético'].isna() ]['dcr_material_genetico'])

# set de códigos de material genético que não tem no datawhere house mas possuí apontamento no boletim
cd_material_vazio = set(mtg_vazio[mtg_vazio['cd_material_genetico'].isna()]['CD_MATERIAL_GENETICO'])

In [527]:
#Criando um data frame de dadas de preparo

data_de_preparo = boletim[['Data Preparo referência pre','cd_uso_solo','objeto de locação','Status']]
data_de_preparo = data_de_preparo[data_de_preparo['Data Preparo referência pre'].notna()].drop_duplicates('cd_uso_solo')


In [528]:
#Insumos fordor para a regra de pre plantio utilizando o fordor

#Agora pode apontar qualquer insumo até 5 depois do plantio 

#Apenas histórico, a regra mudou para qualquer insumo  com características de Pre

insumos_pre_plantio = boletim[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO']]

insumos_pre_plantio = insumos_pre_plantio.merge(insumos[['Fordor','Insumo']], left_on='NOME_RECURSO', right_on='Insumo',how='left').drop('Insumo', axis=1)

insumos_pre_plantio = insumos_pre_plantio[insumos_pre_plantio['Fordor'] == 'Fordor']

In [529]:
# 1º Controle pré

id_controle_pre_plantio = boletim[(boletim['Grupo de Operação'] == 'Herbicida') | (boletim['Grupo de Operação'] == 'Preparo')]

id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['Operação Vigente'] == 'Sim']

id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['QUANTIDADE'] > 0 ]


agg_1_controle = {'QUANTIDADE': 'sum', 'DATA_OPERACAO':'max', 'VLR_PRODUCAO': 'sum'}

id_controle_pre_plantio = id_controle_pre_plantio.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Característica','Grupo de Operação']).agg(agg_1_controle).reset_index()

id_controle_pre_plantio = id_controle_pre_plantio.merge(regua_operacao,right_on='operação', left_on='DCR_OPERACAO',how='left')

#Realizando filtros para ter somente insumos de Pré e que a regua de operação seja diferente de barra
id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['Característica']=='Pré')]
id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['descrição_regua'] !='Barra')]


#Importante definir o que é Pré e Pós preparo
id_controle_pre_plantio = id_controle_pre_plantio.merge(data_de_preparo, left_on='cd_uso_solo', right_on='cd_uso_solo', how='left').drop('Status', axis=1)
id_controle_pre_plantio['Pré-Pós Preparo'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data Preparo referência pre']) else "Pós Preparo" if  x['DATA_OPERACAO'] >= x['Data Preparo referência pre']  else "Pré Preparo", axis=1)

#Importante definir o que é Pré e Pós plantio
id_controle_pre_plantio = id_controle_pre_plantio.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']],left_on='cd_uso_solo', right_on='cd_uso_solo',how='left')
id_controle_pre_plantio['Pré-Pós Plantio'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)



#Problema com talhões sem data de preparo

#Esse Filtro gera o problema quando não temos data de preparo para os talhões, uma opção é alterar para != "Pre Preparo", mas necessário entender quais são os efeitos colaterais


#Removendo talhões que são pós preparo
id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['Pré-Pós Preparo'] != 'Pré Preparo')]



#Encontrar a dose aplicada
id_controle_pre_plantio['Dose/ha 1º pre'] = id_controle_pre_plantio['QUANTIDADE'] / id_controle_pre_plantio['VLR_PRODUCAO']

#Concaternar todos os insumos apontados em um boletim
id_controle_pre_plantio['Insumos apontados 1º pre'] = id_controle_pre_plantio.groupby(['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO'])['NOME_RECURSO'].transform(lambda x:'; '.join(x))

id_controle_pre_plantio['Dose/ha 1º pre'] = id_controle_pre_plantio.groupby(['CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO'])['Dose/ha 1º pre'].transform(lambda x: '; '.join(x.astype(str)))




#Removendo boletins duplicados para poder ordená-los e ter apenas um id por boletim
id_controle_pre_plantio = id_controle_pre_plantio.drop_duplicates('CD_BOLETIM_SILVICULTURA')


#Ordenando a base de forma ascendente
id_controle_pre_plantio = id_controle_pre_plantio.sort_values(['cd_uso_solo','DATA_OPERACAO'],ascending=[True,False])



#Trazendo os insumos que são Fordor
id_controle_pre_plantio = id_controle_pre_plantio.merge(insumos_pre_plantio[['CD_BOLETIM_SILVICULTURA','Fordor']], left_on='CD_BOLETIM_SILVICULTURA', right_on='CD_BOLETIM_SILVICULTURA',how='left')


numero_de_dias_apos_plantio = 5


#A operação precisa ser Pós preparo, e que seja 1º Pré com insumos que tenham características de Pré

id_controle_pre_plantio['gatilho_database'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data de Plantio']) else  
                                                        "1º Pré com insumo pre" if x['Pré-Pós Plantio'] == "Pós Plantio" and (x['DATA_OPERACAO'] - x['Data de Plantio']).days <= numero_de_dias_apos_plantio and x['Característica'] == 'Pré' else 
                                                        'Data Forçada' if x['DATA_OPERACAO'] > x['Data de Plantio'] and x['Grupo de Operação'] == 'Preparo' else 
                                                        "Pós Plantio" if x['DATA_OPERACAO'] > x['Data de Plantio'] else 
                                                        None , axis=1)



#Tudo que for diferente de pós plantio é selecionado
id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['gatilho_database'] != 'Pós Plantio']






#Criando a regra para analisar a ordem das operações
id_controle_pre_plantio['Regra de mudança'] = (id_controle_pre_plantio['cd_uso_solo'] != id_controle_pre_plantio['cd_uso_solo'].shift(1)) & (id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'] != id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'].shift(1))

id_controle_pre_plantio['grupos_distintos'] = id_controle_pre_plantio['Regra de mudança'].cumsum()

id_controle_pre_plantio['Sequência de Operação'] = id_controle_pre_plantio.groupby('grupos_distintos').cumcount()+1



#Necessário considerar apenas o primeiro apontamento referente a operação
id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['Sequência de Operação'] == 1]

#Criando referência do boletim
id_controle_pre_plantio['Boletim 1º pre'] = id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA']

id_controle_pre_plantio = id_controle_pre_plantio.rename({'DATA_OPERACAO':'Data 1º Pre', 'Sequência de Operação':'Pre 1º Operação'},axis=1)

#id_controle_pre_plantio.to_excel('id_controle_pre_plantio.xlsx')


In [530]:
#2º Controle pré

id_controle_pre = boletim[boletim['Grupo de Operação'] == 'Herbicida']

id_controle_pre = id_controle_pre[id_controle_pre['Operação Vigente'] == 'Sim']

id_controle_pre = id_controle_pre[id_controle_pre['QUANTIDADE'] > 0]


#Regra para agrupar. Lembrando que mesmo a data sendo Máxima, agrupo por boletim, então a data sempre será a do boletim.
agg_2_controle = {'QUANTIDADE':'sum', 'DATA_OPERACAO': 'max', 'VLR_PRODUCAO':'sum'}

id_controle_pre = id_controle_pre.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Característica']).agg(agg_2_controle).reset_index()

id_controle_pre = id_controle_pre.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']] , right_on='cd_uso_solo', left_on='cd_uso_solo', how='left')

id_controle_pre['Pré-Pós Plantio'] = id_controle_pre.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)

id_controle_pre = id_controle_pre.merge(regua_operacao, right_on='operação', left_on='DCR_OPERACAO', how='left').drop('operação',axis=1)

id_controle_pre = id_controle_pre.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])


#Filtrar somente o que é Pós plantio

#1º A operação é pós plantio? Preciso filtrar apenas o que é pós
id_controle_pre = id_controle_pre[id_controle_pre['Pré-Pós Plantio'] == "Pós Plantio"]

#Removendo tudo o que eu já tenho boletim no 1ºControle Pré
id_controle_pre = id_controle_pre[ ~ id_controle_pre['CD_BOLETIM_SILVICULTURA'].isin(id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'])]

#2º Se a operação tiver o Nome de 2º Controle -> Devo Zerar a operação, sendo assim só posso trazer o que é 1º Controle ou o que é apontado como pré 
regua_id_2_controle_pre = (id_controle_pre['descrição_regua'] == 'Prés')

id_controle_pre = id_controle_pre[regua_id_2_controle_pre]


#3º Tem insumo de Pré? garantir que  a operação tenha sido apontada de forma correta, que ela tenha insumos de pré

#Mudança de Premissas -> Todas as operações pre emergentes deverão ser contadas e só devemos utilizar as duas primeiras -> A operação não dependerá apenas do nome dela agora

insumos_pre = (id_controle_pre['Característica'] == "Pré")

id_controle_pre = id_controle_pre[insumos_pre]

#Dose de pre
id_controle_pre['Dose/ha pre'] = id_controle_pre['QUANTIDADE'] / id_controle_pre['VLR_PRODUCAO']

id_controle_pre['Insumos apontados pre'] = id_controle_pre.groupby(['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO'])['NOME_RECURSO'].transform(lambda x: '; '.join(x))

id_controle_pre['Dose/ha pre'] = id_controle_pre.groupby(['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO'])['Dose/ha pre'].transform(lambda x: '; '.join(x.astype(str)))




id_controle_pre = id_controle_pre.drop_duplicates('CD_BOLETIM_SILVICULTURA')

id_controle_pre = id_controle_pre.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Definindo uma regra de operações para a mudança de uma talhão para o outro
id_controle_pre['Regra de Mudança'] = (id_controle_pre['CD_BOLETIM_SILVICULTURA'] != id_controle_pre['CD_BOLETIM_SILVICULTURA'].shift(1)) & (id_controle_pre['cd_uso_solo'] != id_controle_pre['cd_uso_solo'].shift(1))

# Criar uma coluna para indicar grupos distintos
id_controle_pre['grupos_distintos'] = id_controle_pre['Regra de Mudança'].cumsum()

#Aplica a soma acumulada para cada grupo distinto
id_controle_pre['Sequência de Operação'] = id_controle_pre.groupby('grupos_distintos').cumcount()+1

#Base de sequencia de operações completa ->Solicitação Gabi
base_sequencia_controle_pre = id_controle_pre.copy()

#Filtrando o que tem somente até segunda operação, o restante das operações deverá ser desconsiderado
id_controle_pre = id_controle_pre[id_controle_pre['Sequência de Operação'] <= 2]

#Ajustando a ordem de operação {1:2,2:3}
id_controle_pre['Sequência de Operação'] = id_controle_pre['Sequência de Operação']+1 



#Inserir ajuste de Sequencia
id_controle_pre = id_controle_pre.merge(sequencia_vigente[['boletim','Operação Anterior', 'Operação Atual','Sequência Vigente']], left_on='CD_BOLETIM_SILVICULTURA' , right_on='boletim', how='left').drop('boletim',axis=1)
#Quando a sequência vigente não for nula, ela assume a nova sequência de operação, caso contrário ela segue vida
id_controle_pre['Sequência de Operação'] = id_controle_pre.apply(lambda x: x['Sequência Vigente'] if pd.notna(x['Sequência Vigente']) else x['Sequência de Operação'], axis=1)



#Fazendo Referência no boletim 
id_controle_pre['Boletim pre'] = id_controle_pre['CD_BOLETIM_SILVICULTURA']


id_controle_pre = id_controle_pre.rename({'DATA_OPERACAO':'Data Pre','Sequência de Operação':'Pre'},axis=1)


In [531]:
#id_controle_pre.to_excel('id_controle_pre.xlsx')

In [532]:
#Regras de controles pre

#Não posso utilizar boletins repetidos dentro das outras operações -> Boletins dos Controles pre

#Não posso utilizar boletins repetidos dentro das outras operações -> Boletins dos Controles pre plantio

#Na barra não terá boletins repetidos devido aos filtros de Pós Plantio e Característica de Pós

In [533]:
#Barra 


barra = boletim[((boletim['Grupo de Operação'] == 'Herbicida') | (boletim['Grupo de Operação'] == 'Adubação com herbicida'))] 

barra = barra[barra['Operação Vigente'] == 'Sim']

agg_barra = {'QUANTIDADE': 'sum', 'DATA_OPERACAO':'max', 'VLR_PRODUCAO':'sum'}

barra = barra[barra['QUANTIDADE'] > 0]

#Boletim já é filtrado para operação e insumo vigente e apontamentos que não são cancelados e reprovados

barra = barra.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Grupo de Operação','Operação Vigente']).agg(agg_barra).reset_index()

barra = barra.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

barra = barra.merge(insumos[['Insumo','Insumo Vigente','Característica pre e pos','Característica']], left_on='NOME_RECURSO', right_on='Insumo', how='left')

barra = barra.merge(regua_operacao,left_on='DCR_OPERACAO', right_on='operação',how='left').drop('operação',axis=1)

barra['Pré-Pós Plantio'] = barra.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)


#Filtrando somente operação que é após plantio 
barra = barra[barra['Pré-Pós Plantio'] == 'Pós Plantio']

#Filtrando característica do insumo # Ou Pré e Pós
#Ajustar na planilha de premissas os herbicidas para duplicarem ->Impacto de duplicar a base
barra = barra[barra['Característica'] == 'Pós']

#Dose Barra
barra['Dose/ha barra'] = barra['QUANTIDADE'] / barra['VLR_PRODUCAO']

barra['Insumos apontados barra'] = barra.groupby(['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO'])['NOME_RECURSO'].transform(lambda x: '; '.join(x))

barra['Dose/ha barra'] = barra.groupby(['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO'])['Dose/ha barra'].transform(lambda x: '; '.join(x.astype(str)))


#Definindo coluna talhão para filtrar os 300
barra['Talhão'] = barra['objeto de locação'].str[-3:]

#Filtrando a descrição da operação e talhões 300, que podem ser talhões manuais
barra = barra[(barra['descrição_regua'] == 'Barra') | (barra['Talhão'] == '300')]

#Se a operação de barra for maior do que x dias após o plantio, não considerar
dias_barra_e_plantio = 500
barra['dias_barra_plantio'] = barra.apply(lambda x:  x['DATA_OPERACAO'] - x['Data de Plantio'] if pd.notna(x['DATA_OPERACAO']) else None, axis=1)
barra = barra[barra['dias_barra_plantio'] <= pd.Timedelta(days=dias_barra_e_plantio)]


#Remover duplicatas de boletim, devido a presença de mais de um insumo por boletim
barra = barra.drop_duplicates('CD_BOLETIM_SILVICULTURA')

#Ordenar por objeto de locação e por data
barra = barra.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

barra['Regra de mudança'] = (barra['cd_uso_solo'] != barra['cd_uso_solo'].shift(1)) & (barra['CD_BOLETIM_SILVICULTURA'] != barra['CD_BOLETIM_SILVICULTURA'].shift(1))

barra['grupos_distintos'] = barra['Regra de mudança'].cumsum()

barra['Sequência de Operação'] = barra.groupby('grupos_distintos').cumcount()+1

#Filtrar somente as três primeiras operações (1º, 2º e 3º  Barra)
barra= barra[barra['Sequência de Operação'] <=3]


#Inserir ajuste de Sequencia

barra = barra.merge(sequencia_vigente[['boletim','Operação Anterior', 'Operação Atual','Sequência Vigente']], left_on='CD_BOLETIM_SILVICULTURA' , right_on='boletim', how='left').drop('boletim',axis=1)

#Quando a sequência vigente não for nula, ela assume a nova sequência de operação, caso contrário ela segue vida
barra['Sequência de Operação'] = barra.apply(lambda x: x['Sequência Vigente'] if pd.notna(x['Sequência Vigente']) else x['Sequência de Operação'], axis=1)


#Fazendo Referência no boletim 
barra['Boletim Barra'] = barra['CD_BOLETIM_SILVICULTURA']

#Renomeando as colunas
barra = barra.rename({'DATA_OPERACAO':'Data Barra','Sequência de Operação':'Barra'},axis=1)

#barra.to_excel("barra.xlsx")

In [534]:
#Calcário

#insumos maior que zero;
calcario = boletim[boletim['QUANTIDADE'] > 0]

#Filtrando o boletim apenas para o grupo de operações de adubação
calcario = calcario[(calcario['Grupo de Operação'] == 'Calcário') | (calcario['Grupo de Operação'] == 'Adubação')]

calcario = calcario[calcario['Operação Vigente'] == 'Sim']


#Regra para agrupar. Lembrando que mesmo a data sendo Máxima, agrupo por boletim, então a data sempre será a do boletim.
agg_adub = {'QUANTIDADE':'sum', 'DATA_OPERACAO': 'max'}

calcario = calcario.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Grupo de Operação','Operação Vigente']).agg(agg_adub).reset_index()

#Merge para a área do talhão utilizando o cd_uso_solo, assim garante a área que já foi apontada no talhão original
calcario = calcario.merge(cadastro_ciclo[['cd_uso_solo','vlr_area']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

calcario['dose/ha'] = calcario['QUANTIDADE'] / calcario['vlr_area']


calcario = calcario.merge(insumos[['Insumo','Característica','Insumo Vigente','Formulação']],left_on='NOME_RECURSO',right_on='Insumo',how='left').drop('Insumo',axis=1)

#Apenas características de insumo 
calcario = calcario[(calcario['Característica'] == 'Calcário') | (calcario['Característica'] == 'Oxyfertil')]

#Apenas Insumo vigente

calcario = calcario[calcario['Insumo Vigente'] == 'Sim']

calcario = calcario.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo',how='left')

calcario['delta_operacao'] = (pd.to_datetime(calcario['DATA_OPERACAO']) - pd.to_datetime(calcario['Data de Plantio'])).dt.days

#A operação de calcário pode acontecer até 420 dias depois do plantio e um ano antes, e a complementar somente após



min_delta_calcario = -365
max_delta_calcario = 365

calcario = calcario[(calcario['delta_operacao'] <= max_delta_calcario) & (calcario['delta_operacao'] >= min_delta_calcario)]



#Necessário criar um DF auxiliar, devido ao insumos duplicados dentro de um mesmo boletim
calcario_regra_de_mudanca = calcario.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA')
#Ordenar o data frame

calcario_regra_de_mudanca = calcario_regra_de_mudanca.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Realizando a regra de mudança de cd_uso_solo e boletim
calcario_regra_de_mudanca['regra de mudança'] = (calcario_regra_de_mudanca['cd_uso_solo'] != calcario_regra_de_mudanca['cd_uso_solo'].shift(1)) & (calcario_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'] != calcario_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
#Realizando a soma acumulada
calcario_regra_de_mudanca['grupos_distintos'] = calcario_regra_de_mudanca['regra de mudança'].cumsum()
#Criando a ordem de operação
calcario_regra_de_mudanca['Sequencia de Operação'] = calcario_regra_de_mudanca.groupby('grupos_distintos').cumcount()+1
#Trazendo a ordem da operação para adubação novamente
calcario = calcario.merge(calcario_regra_de_mudanca[['Sequencia de Operação','CD_BOLETIM_SILVICULTURA']], left_on='CD_BOLETIM_SILVICULTURA', right_on='CD_BOLETIM_SILVICULTURA', how='left')

#Somando o volume aplicado 
calcario['Dose/ha acumulada calagem'] = calcario.groupby('cd_uso_solo')['dose/ha'].transform('sum')


#Definir se a operação é pré ou pós plantio
calcario['status_calagem'] = calcario.apply(lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else 'Pré Plantio', axis=1)

#Número de avaliações realizadas no mesmo cd_uso solo
calcario['numero_avaliacoes_calagem'] = calcario.groupby('cd_uso_solo')['cd_uso_solo'].transform('count')

#Contagem de status_calagem por operação apontados por cd_uso_solo
calcario['vlr_status_calagem'] = calcario.groupby(['cd_uso_solo','status_calagem'])['cd_uso_solo'].transform('count')


#Ordenar novamente o data frame de calcário para criar uma regra de filtrar as operações
calcario = calcario.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Contagem de operações realizadas dentro do cd_uso_solo sendo pre_pos_calagem
calcario['pre_pos_calagem'] = calcario.groupby('cd_uso_solo')['status_calagem'].transform('nunique')

#Trazendo recomendação de calcario
calcario = calcario.merge(recomendacao_adu[['Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação','Objeto de locação']], left_on='objeto de locação', right_on='Objeto de locação',how='left').drop('Objeto de locação',axis=1).rename({'Data de recomendação':'Data de recomendação calagem'},axis=1)


#gerando regra para considerar a operação 


calcario['regra_operacao_calagem'] = np.where(
    calcario['numero_avaliacoes_calagem'] == 1, 'Considerar 1 op',
    np.where(
        (calcario['status_calagem'] == 'Pré Plantio') &
        (calcario['numero_avaliacoes_calagem'] >= 2) &
        (calcario['numero_avaliacoes_calagem'] == calcario['Sequencia de Operação']),
        'considerar pre',
        np.where(
            (calcario['status_calagem'] == 'Pós Plantio') & 
            (calcario['numero_avaliacoes_calagem'] >= 2) & 
            (calcario['Sequencia de Operação'] == 1),
            'considerar pos',
            np.where(
                (calcario['pre_pos_calagem'] == 2) &
                (calcario['status_calagem'] == 'Pré Plantio') &
                (calcario['vlr_status_calagem'] == calcario['Sequencia de Operação']),
                'considerar pre max sequencia',
                'Não considerar'
            )
        )
    )
)


#calcario.to_excel('calcario.xlsx')


#Filtrando somente as operações para considerar
calcario = calcario[calcario['regra_operacao_calagem'] != 'Não considerar']


#Sempre dar preferência ao pré plantio

#Observando onde temos talhões iguais para as mesmas regras de operação
#Se for considerar apenas a primeira que acontecer, a fórmula está certa. Caso contrário só alterar o shift para -1 que levará o true para o local correto
calcario['status_calagem_dup'] = ((calcario['cd_uso_solo'] == calcario['cd_uso_solo'].shift(1)) & (calcario['regra_operacao_calagem'] == calcario['regra_operacao_calagem'].shift(1)) 
                                 | (calcario['cd_uso_solo'] == calcario['cd_uso_solo'].shift(1)) & (calcario['regra_operacao_calagem'] != calcario['regra_operacao_calagem'].shift(1)) )

#Removendo onde temos classificações iguais para calagem
calcario = calcario[calcario['status_calagem_dup'] != True]





calcario['Boletim Calagem'] = calcario['CD_BOLETIM_SILVICULTURA']

calcario = calcario.rename({'DATA_OPERACAO':'Data Calagem', 'NOME_RECURSO': 'Insumo calagem' , 'numero_avaliacoes_calagem': 'Número de Avaliações','Sequencia de Operação':'Sequência Calagem', 'regra_operacao':'regra_calagem'},axis=1)



In [535]:
#Adubação de base

#Filtrando apontamento de insumo maior que 0
base = boletim[boletim['QUANTIDADE'] > 0]

#Filtrando grupo de operação (Preparo e Adubação) e operação vigente
base = base[((base['Grupo de Operação'] == 'Adubação')| (base['Grupo de Operação'] == 'Preparo') ) & (base['Operação Vigente'] == 'Sim')] 

#Filtrando tipo de operação
base = base[base['Tipo de operação'] !='Cobertura']


#Filtrando a característica e insumo vigente
base = base[((base['Característica'] == 'Adubo') | (base['Característica'] == 'Oxyfertil')) & (base['Insumo Vigente'] == 'Sim')]

#Filtrando o tipo de operação de adubação -> Base
base = base[(base['Tipo de Adubação'] == 'Base') | (base['Tipo de Adubação'] == 'Calagem')  | (base['Tipo de Adubação'] == 'Manutenção') ]

#Delta plantio em relação a operação
base['Delta de plantio'] = (base['DATA_OPERACAO'] - base['Data de Plantio']).dt.days

#Filtrando apenas o que poderia estar dentro do range máximo e mínimo de operação

#min = 90 e max = 120
base_min = -90
base_max = 150
base = base[(base['Delta de plantio'] > base_min) & (base['Delta de plantio'] < base_max)]

#Classificar o data frame por 'cd_uso_solo' e data de operação e a quantidade, assim garante que o insumo válido será sempre o de maior volume
base = base.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[True,True,False])


#retirar os boletins de calagem
base = base[ ~ base['CD_BOLETIM_SILVICULTURA'].isin(calcario['CD_BOLETIM_SILVICULTURA'])]


#Criar uma cópia do boletim para excluir os boletins duplicados, e gerar a ordem de operação 1 e 2, com isso trazer devolta para a operação de base
base_regra_de_mudanca = base.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')

base_regra_de_mudanca['regra_de_mudanca'] = (base_regra_de_mudanca['cd_uso_solo'] != base_regra_de_mudanca['cd_uso_solo'].shift(1)) & (base_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'] != base_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
#Realizando a soma acumulada
base_regra_de_mudanca['grupos_distintos'] = base_regra_de_mudanca['regra_de_mudanca'].cumsum()
#Criando a ordem de operação
base_regra_de_mudanca['Sequência base'] = base_regra_de_mudanca.groupby('grupos_distintos').cumcount()+1

#trazer para a base a sequencia de operação
base = base.merge(base_regra_de_mudanca[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência base']], left_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], how='left')

#Dose aplicada
base['Dose/ha base'] = base['QUANTIDADE'] / base['VLR_PRODUCAO']

#Dose aplicada acumulada
base['Dose/ha acumulada base'] = base.groupby(['cd_uso_solo'])['Dose/ha base'].transform('sum')


#Classificar o que é pré e pós plantio 
#Definir ordem para o que é pré e pós plantio

base['status_base'] = base.apply(lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else 'Pré Plantio',axis=1)

#Encontrando o número de operações para o talhão
base['numero de avaliacoes base'] = base.groupby(['cd_uso_solo'])['cd_uso_solo'].transform('count')

#Contagem de status apontado por 'cd_uso_solo'
base['vlr_status_base'] = base.groupby(['cd_uso_solo','status_base'])['cd_uso_solo'].transform('count')

#Contagem distinta do número de status em um 'cd_uso_solo'
base['pre_pos_base'] = base.groupby('cd_uso_solo')['status_base'].transform('nunique')


#Trazendo a recomendação 

base = base.merge(recomendacao_adu[['Insumo base','Dose kg/ha base','Data de recomendação','Objeto de locação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação',axis=1).rename({'Data de recomendação':'Data de recomendação base', 'Insumo base':'Insumo base - rec','Dose kg/ha base':'Dose kg/ha base - rec'},axis=1)

#Definindo valor aplicado para reforço
volume_reforço = 120


data_reforco = 60


#Se já tem base tem apontamento de adubação até 60 dias ele é reforço


#Gerando regra para considerar a operação

base['regra_operacao_base'] = np.where((base['numero de avaliacoes base'] == 1) & 
                                       (base['Sequência base'] ==1) , 
                                       'Considerar 1 Op  & > insumo',
                                       np.where((base['status_base'] == 'Pré Plantio') &
                                                (base['numero de avaliacoes base'] >= 2)&
                                                (base['numero de avaliacoes base'] == base['Sequência base']),'Considerar Pré',
                                                np.where((base['status_base']== 'Pós Plantio')&
                                                         (base['numero de avaliacoes base']>=2) &
                                                         (base['Sequência base'] ==1), 'Considerar Pós',
                                                         np.where((base['pre_pos_base'] == 2)&
                                                                  (base['status_base']== 'Pré Plantio')&
                                                                  (base['vlr_status_base'] == base['Sequência base']), 'considerar pre max sequencia',
                                                                  np.where( (base['Sequência base'] >= 2)&
                                                                            (base['status_base'] == 'Pós Plantio')&
                                                                            (base['Formulação'] == 'Oxyfertil') , 'Reforço',        
                                                                        np.where((base['Sequência base'] == 2) &
                                                                                (base['status_base'] == 'Pós Plantio')&
                                                                                (base['Dose/ha base'] <= volume_reforço), 'Reforço', 
                                                                                np.where(
                                                                                    (base['Sequência base'] >=2)&
                                                                                    (base['status_base'] == 'Pós Plantio')&
                                                                                    (base['Delta de plantio'] <= data_reforco), 'Reforço2',
                                                                                        'Não considerar'
                                                                                         )                                                                                                                                       
                                                                                )
                                                                        )                                                               
                                                                  )
                                                        )
                                                )
                                        )


#Se eu tiver um talhão com considerar pré e um talhão com considerar pós, devo sempre utilizar o considerar Pré!

base['unica_operacao_base'] = ( ((base['cd_uso_solo'] == base['cd_uso_solo'].shift(1)) & (base['regra_operacao_base'] == 'Considerar Pós') & (base['regra_operacao_base'].shift(1) == 'Considerar Pré')) |
                               
((base['cd_uso_solo'] == base['cd_uso_solo'].shift(1)) & (base['regra_operacao_base'] == 'Considerar Pós') & (base['regra_operacao_base'].shift(1) == 'considerar pre max sequencia')) |

((base['cd_uso_solo'] == base['cd_uso_solo'].shift(1))  &  (base['regra_operacao_base'] == base['regra_operacao_base'].shift(1)))

) 



base = base[base['unica_operacao_base'] != True]


#Exportar base bruta com todas as informações para chegar no resultado
#base.to_excel('base.xlsx')

#Filtrar apenas operações que devem ser consideradas ao levar a informação para o boletim

base = base[(base['regra_operacao_base'] != 'Não considerar') & (base['regra_operacao_base'] != 'Reforço2') ]

#Fazendo Referência no boletim 
base['Boletim Base'] = base['CD_BOLETIM_SILVICULTURA']


base = base.rename({'DATA_OPERACAO':'Data Adub. Base','Estratégia':'Estratégia Base', 'NOME_RECURSO':'Insumo Adubação base' },axis=1)



#criar uma lógica para cob1 e cob2 não ter os id's que já considerei na base



In [536]:
#Estratégia de adubação

estrategia = base[['cd_uso_solo','Estratégia Base','QUANTIDADE','regra_operacao_base','Insumo Adubação base']]

#O que define a estratégia é o insumo e a sua quantidade determina qual será a do talhão
estrategia = estrategia.sort_values(['cd_uso_solo','QUANTIDADE'], ascending=[True,False])

estrategia = estrategia.drop_duplicates('cd_uso_solo',keep='first')

#estrategia.to_excel('estrategia.xlsx')


#Quando não tenho apontamentos de base o que define a estratégia é a recomendação de base



In [537]:
#Adubações de cobertura 


adu_cob = boletim[boletim['QUANTIDADE'] >0]


#Filtro de grupo de operação -> Adubação e Operação vigente -> Sim
adu_cob = adu_cob[((adu_cob['Grupo de Operação']=='Adubação') | (adu_cob['Grupo de Operação']=='Adubação com herbicida') ) & (adu_cob['Operação Vigente'] == 'Sim')]

adu_cob = adu_cob[adu_cob['Tipo de operação'] != 'Base']



adu_cob = adu_cob[(adu_cob['Tipo de Adubação'] != 'Base')] 


#Filtrando grupo de insumos -> Caracteristica Adubo e Insumo vigente ->
adu_cob = adu_cob[((adu_cob['Característica'] == 'Adubo') | (adu_cob['Característica'] == 'Oxyfertil') ) & (adu_cob['Insumo Vigente'] == 'Sim')]

#Retirar os apontamentos de base em adu_cob -> Assim garante que não teremos apontamentos duplicados
adu_cob = adu_cob[ ~ adu_cob['CD_BOLETIM_SILVICULTURA'].isin(base['CD_BOLETIM_SILVICULTURA'])]

#Retirar os apontamentos de calagem em adu_cob 
adu_cob = adu_cob[ ~ adu_cob['CD_BOLETIM_SILVICULTURA'].isin(calcario['CD_BOLETIM_SILVICULTURA'])]


#Trazendo estratégia executada pela operação
adu_cob = adu_cob.merge(estrategia[['cd_uso_solo','Estratégia Base']], left_on='cd_uso_solo', right_on='cd_uso_solo' , how='left')


#Trazendo a estratégia recomendada pelo P&D
adu_cob = adu_cob.merge(recomendacao_adu[['Objeto de locação','Estratégia Recomendada', 'Insumo cobertura','Dose kg/ha cobertura','Insumo cobertura 2', 'Dose kg/ha cobertura 2','Data de recomendação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação', axis=1).rename({'Insumo cobertura':'Insumo cobertura - rec','Dose kg/ha cobertura':'Dose kg/ha cobertura - rec', 'Insumo cobertura 2':'Insumo cobertura 2 - rec','Dose kg/ha cobertura 2':'Dose kg/ha cobertura 2 - rec', 'Data de recomendação': 'Data de recomendação cobertura'},axis=1)


#Trazendo a recomendação de complementar para a base
adu_cob = adu_cob.merge(recomendacao_comp[['Objeto de locação','NPK complementar','Dose kg/ha complementar','Corretivos complementares','Dose kg/ha corretivos','Data de recomendação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação', axis=1).rename({'Dose kg/ha corretivos':'Dose kg/ha corretivos - rec','Dose kg/ha complementar': 'Dose kg/ha complementar - rec'},axis=1)


#Estratégia ajustada

adu_cob['Estratégia ajustada'] = adu_cob.apply(lambda x: x['Estratégia Recomendada'] if pd.isna(x['Estratégia Base']) else x['Estratégia Base'],axis=1)


#Definindo o status se é pré ou pós plantio
adu_cob['Status adu_cob'] = adu_cob.apply( lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else None if pd.isna(x['Data de Plantio']) else 'Pré Plantio', axis=1)


#Coluna de delta de plantio
adu_cob['Delta de plantio adu_cob'] = (adu_cob['DATA_OPERACAO'] - adu_cob['Data de Plantio']).dt.days


#Filtrar somente o que é pós plantio
adu_cob = adu_cob[adu_cob['Status adu_cob'] == 'Pós Plantio']

adu_cob = adu_cob.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[True,True,False])


#Dose aplicada
adu_cob['Dose/ha adu cob'] = adu_cob['QUANTIDADE'] / adu_cob['VLR_PRODUCAO']

#Dose aplicada acumulada
adu_cob['Dose/ha acumulada adu cob'] = adu_cob.groupby(['cd_uso_solo'])['Dose/ha adu cob'].transform('sum')


#Concaternar todos os insumos e formulações de um boletim para ter todos em uma mesma lista
adu_cob['Insumos apontados'] = adu_cob.groupby(['CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO',])['NOME_RECURSO'].transform(lambda x: ', '.join(x))

#Preenchendo campos vazios em formulação para não perder informação e apresentar erros no código

adu_cob['Formulação'] = adu_cob['Formulação'].astype(str).fillna('')
adu_cob['Formulações apontadas'] = adu_cob.groupby(['CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO',])['Formulação'].transform(lambda x: ', '.join(x))


#Ordernar o data frame -> tenho mais de um insumo apontado duplicando o boletim, preciso criar um dataframe intermediário
#O insumo que teve mais apontamento será o considerado para a estratégia
adu_cob_regra_mudanca = adu_cob.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')
adu_cob_regra_mudanca['regra de mudança adu_cob'] = (adu_cob_regra_mudanca['cd_uso_solo'] != adu_cob_regra_mudanca['cd_uso_solo'].shift(1)) & (adu_cob_regra_mudanca['CD_BOLETIM_SILVICULTURA'] != adu_cob_regra_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
adu_cob_regra_mudanca['grupos distindos adu_cob'] = adu_cob_regra_mudanca['regra de mudança adu_cob'].cumsum()
#Sequência de operação
adu_cob_regra_mudanca['Sequência Cob'] = adu_cob_regra_mudanca.groupby('grupos distindos adu_cob').cumcount()+1



#Trazendo a sequencia de operações  da base de adubação de cobertura regra de mudança
adu_cob = adu_cob.merge(adu_cob_regra_mudanca[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência Cob']], left_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], how='left')


#Inserir ajuste de Sequência
adu_cob = adu_cob.merge(sequencia_vigente[['boletim','Operação Anterior','Operação Atual','Sequência Vigente']], left_on='CD_BOLETIM_SILVICULTURA', right_on='boletim', how='left',suffixes=('',' - Adubação')).drop('boletim', axis=1)

#Quando a sequência vigente não for nula, ela assume a nova sequência de operação, caso contrário ela segue vida
adu_cob['Sequência Cob'] = adu_cob.apply(lambda x: x['Sequência Vigente'] if pd.notna(x['Sequência Vigente']) else x['Sequência Cob'] , axis=1)



#Se a data de recomendação for maior que a data de operação, não devo considerar que houve recomendação
adu_cob['Existe recomendação'] = adu_cob.apply(lambda x: "Não" if ((x['Corretivos complementares'] == 'n.a.') | pd.isna(x['Corretivos complementares'])) & 
                                              ((x['NPK complementar'] == 'n.a.') | pd.isna(x['NPK complementar'])) |
                                              (x['DATA_OPERACAO'] < x['Data de recomendação'])
                                              else 'Sim',axis=1)


adu_cob['recomendação comp'] = adu_cob.apply(lambda x: "Sem recomendação" if ((x['Corretivos complementares'] == 'n.a.') | pd.isna(x['Corretivos complementares'])) & 
                                              ((x['NPK complementar'] == 'n.a.') | pd.isna(x['NPK complementar'])) else "Aderente" 
                                              if (x['NPK complementar'] in x['Formulações apontadas'].split(', ')) |
                                             (x['Corretivos complementares'] in x['Formulações apontadas'].split(', ')) else 'Não Aderente',
                                               axis=1)

delta_complementar = 450

adu_cob['regra_operação'] = np.where(adu_cob['Sequência Cob'] >= 5, "Desconsiderar",
                                    np.where((adu_cob['Sequência Cob'] == 1) & 
                                             (adu_cob['Tipo de Adubação'] != 'Calagem'), 'Cob 1',
                                        np.where(
                                            (adu_cob['Estratégia ajustada'] == 'B') & 
                                            (adu_cob['Sequência Cob'] == 2) &
                                            (adu_cob['Tipo de Adubação'] != 'Calagem'), 'Cob 2',
                                                np.where( (adu_cob['Estratégia ajustada'] == 'B') &
                                                        ( adu_cob['Sequência Cob'] >2) & 
                                                        (adu_cob['Existe recomendação'] == 'Sim')&
                                                        (adu_cob['recomendação comp'] == 'Aderente'), 'Complementar',
                                                    np.where(
                                                        (adu_cob['Estratégia ajustada'] == 'B')& 
                                                        (adu_cob['Sequência Cob'] > 2) &
                                                        (adu_cob['Delta de plantio adu_cob'] >= delta_complementar), 'Complementar',
                                                        np.where(
                                                            (adu_cob['Estratégia ajustada'] == 'A')&
                                                            (adu_cob['Sequência Cob'] == 2)&
                                                            (adu_cob['Delta de plantio adu_cob'] < delta_complementar) &
                                                            (adu_cob['Existe recomendação'] == 'Não')&
                                                            (adu_cob['Tipo de Adubação'] != 'Calagem'), 'Cob 2',
                                                            np.where(
                                                                (adu_cob['Estratégia ajustada'] == 'A') &
                                                                (adu_cob['Sequência Cob'] >= 2) &
                                                                (adu_cob['Delta de plantio adu_cob'] >= delta_complementar), 'Complementar',
                                                                np.where(
                                                                    (adu_cob['Estratégia ajustada'] == 'A')&
                                                                    (adu_cob['Sequência Cob'] >= 2) &
                                                                    (adu_cob['Delta de plantio adu_cob'] < delta_complementar) &
                                                                    (adu_cob['recomendação comp'] == 'Aderente'), 'Complementar',
                                                                    np.where(
                                                                        (adu_cob['Estratégia ajustada'] == 'A')&
                                                                        (adu_cob['Sequência Cob'] >= 2)&
                                                                        (adu_cob['Delta de plantio adu_cob'] > delta_complementar) , 'Complementar',

                                                                        np.where(
                                                                            (adu_cob['Estratégia ajustada'] == 'A') &
                                                                            (adu_cob['Sequência Cob'] > 2) &
                                                                            (adu_cob['Delta de plantio adu_cob'] < delta_complementar),'Complementar',
                                                                            
                                                                            np.where(
                                                                                (adu_cob['Estratégia ajustada'] == 'T') &
                                                                                (adu_cob['Sequência Cob'] > 2), 'Complementar T', None
                                                                                        )
                                                                                    )
                                                                                )
                                                                            )
                                                                        )
                                                                    ) 
                                                                )
                                                            )
                                                        ) 
                                                    )
                                                )
                                        

adu_cob['Ordem complementar'] = adu_cob[adu_cob['regra_operação'] == 'Complementar'].groupby(['cd_uso_solo','regra_operação'])['cd_uso_solo'].cumcount()+1

adu_cob['regra_operação'] = adu_cob.apply(lambda x: str(x['regra_operação']) + ' ' + str(int(x['Ordem complementar'])) if (x['regra_operação'] == 'Complementar') else x['regra_operação'], axis=1)


#Exportando a base bruta
#adu_cob.to_excel('adu_cob.xlsx')



#Fazendo Referência no boletim 
adu_cob['Boletim Adu cob'] = adu_cob['CD_BOLETIM_SILVICULTURA']

#Filtrando os dados para seguir somente o que realmente será necessário
adu_cob = adu_cob[(adu_cob['regra_operação'] != 'Desconsiderar') & ((adu_cob['Ordem complementar'].isna()) | (adu_cob['Ordem complementar'] <= 2)) & (pd.notna(adu_cob['regra_operação']))]


#Removendo os apontamentos que temos iguais 

#Sempre pegando o último apontamento realizado quando tenho duas classificações iguais
adu_cob['status_adu_cob_dup'] = (adu_cob['cd_uso_solo'] == adu_cob['cd_uso_solo'].shift(-1)) & (adu_cob['regra_operação'] == adu_cob['regra_operação'].shift(-1))

adu_cob = adu_cob[adu_cob['status_adu_cob_dup'] != True]



adu_cob = adu_cob.rename({'DATA_OPERACAO':'Data Adub. cob', 'NOME_RECURSO':'Insumo Cob' },axis=1)


In [538]:
#Preparo de solos

#Trazer para o boletim as informações de Preparo de solos

#Trazer pelo CD_BOLETIM_SILVICULTURA, NOME_RECURSO ,DCR_OPERACAO, cd_uso_solo

boletim = boletim.merge(preparo_solo[['CD_BOLETIM_SILVICULTURA','Boletim Preparo de Solos', 'Data Preparo de Solos', 'DCR_OPERACAO', 'cd_uso_solo', 'regra_de_operacao_preparo_solo','NOME_RECURSO']], left_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], right_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], how='left')

boletim['Preparo de solos'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_de_operacao_preparo_solo']) if x['regra_de_operacao_preparo_solo'] == 'Preparo de Solos' else None, axis=1)



In [539]:
#Savannah

#Trazer para o boletim as informações de Savannah

#Trazer pelo CD_BOLETIM_SILVICULTURA, NOME_RECURSO, DCR_OPERACAO, cd_uso_solo

boletim = boletim.merge(savannah[['CD_BOLETIM_SILVICULTURA','Boletim Savannah','Data Savannah','DCR_OPERACAO', 'NOME_RECURSO', 'cd_uso_solo', 'regra_de_operacao_savannah' ]] , left_on=['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], right_on=['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], how='left')

boletim['Savannah'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_de_operacao_savannah']) if x['regra_de_operacao_savannah'] == 'Savannah' else None , axis=1) 



In [540]:
#Dessecação

#Trazer para o boletim as informações de Dessecação

#Trazer pelo CD_BOLETIM_SILVICULTURA, NOME_RECURSO, DCR_OPERACAO, cd_uso_solo

boletim = boletim.merge(desseca[['CD_BOLETIM_SILVICULTURA', 'Boletim Dessecação','Data de Dessecação' ,'DCR_OPERACAO', 'NOME_RECURSO', 'cd_uso_solo', 'Dose/ha desseca', 'Dose/ha desseca acumulada', 'regra_de_operacao_desseca']], left_on=['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], right_on=['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], how='left')


#Definir as colunas do 1º e 2º Dessecação
boletim['Data 1º Dessecação'] = boletim.apply(lambda x: x['Data de Dessecação'] if x['regra_de_operacao_desseca'] == '1º Dessecação' else None , axis=1)
boletim['1º Dessecação'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_de_operacao_desseca']) if x['regra_de_operacao_desseca'] == '1º Dessecação' else None, axis= 1 )

boletim['Data 2º Dessecação'] = boletim.apply(lambda x: x['Data de Dessecação'] if x['regra_de_operacao_desseca'] == '2º Dessecação' else None , axis=1)
boletim['2º Dessecação'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_de_operacao_desseca']) if x['regra_de_operacao_desseca'] == '2º Dessecação' else None, axis= 1 )



In [541]:
#Formiga Pre plantio

#Trazer para o boletim as informações de Formiga Pré Plantio

#trazer pelo  CD_BOLETIM_SILVICULTURA, NOME_RECURSO, DCR_OPERACAO, cd_uso_solo
boletim = boletim.merge(formiga_pre_plant[['CD_BOLETIM_SILVICULTURA','boletim formiga pre plantio','Data de formiga pre plantio','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo','data_cto_baldeio','Dose/ha form pre plantio', 'Dose acumulada/ha form pre plantio', 'Formiga Pre Plantio','regra_operacao_form_pre_plantio']], left_on=['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], right_on= ['CD_BOLETIM_SILVICULTURA', 'NOME_RECURSO', 'DCR_OPERACAO', 'cd_uso_solo'], how ='left')


#Definir as colunas do 1º e 2º Combate Pre Plantio
boletim['Data 1º Formiga Pre Plantio'] = boletim.apply(lambda x: x['Data de formiga pre plantio'] if x['regra_operacao_form_pre_plantio'] == '1º Formiga Pre Plantio' else None, axis=1)
boletim['1º Formiga Pre Plantio'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_operacao_form_pre_plantio']) if x['regra_operacao_form_pre_plantio'] == '1º Formiga Pre Plantio' else None, axis= 1 )

boletim['Data 2º Formiga Pre Plantio'] = boletim.apply(lambda x: x['Data de formiga pre plantio'] if x['regra_operacao_form_pre_plantio'] == '2º Formiga Pre Plantio' else None, axis=1)
boletim['2º Formiga Pre Plantio'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_operacao_form_pre_plantio']) if x['regra_operacao_form_pre_plantio'] == '2º Formiga Pre Plantio' else None, axis= 1 )



In [542]:
#Replantio

#Trazer para o boletim as informações de Replantio
boletim = boletim.merge(replantio[['CD_BOLETIM_SILVICULTURA','Boletim Replantio','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo','Data Replantio','regra_operacao_replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo','numero de replantios','dcr_material_genetico','abrev_material_genetico','Classe MG Plantio','Material Genético']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], how='left')


#Definir as colunas de 1º replantio e de 2º Replantio 
boletim['Data 1º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '1º Replantio' else None , axis=1)
boletim['1º Replantio'] = boletim.apply(lambda x : str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '1º Replantio' else None , axis=1)



#Definir as colunas de 2º Replantio 
boletim['Data 2º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '2º Replantio' else None , axis=1)
boletim['2º Replantio'] = boletim.apply(lambda x : str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '2º Replantio' else None , axis=1)


#Definir as colunas de 3º Replantio
boletim['Data 3º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '3º Replantio' else None , axis=1)
boletim['3º Replantio'] = boletim.apply(lambda x :  str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '3º Replantio' else None , axis=1)

#Definir a coluna de apoio para replantio
boletim['Replantio_apoio'] = boletim.apply(lambda x :  str(x['cd_uso_solo']) + 'replantio' if (x['regra_operacao_replantio'] == '1º Replantio') | (x['regra_operacao_replantio'] == '2º Replantio') | (x['regra_operacao_replantio'] == '3º Replantio') else None , axis=1)



In [543]:
#1º Controle Pre emergente

#Trazer para o boletim no mesmo formato que trouxemos o 2º e o 3º controle
boletim = boletim.merge(id_controle_pre_plantio[['CD_BOLETIM_SILVICULTURA','Boletim 1º pre','DCR_OPERACAO','NOME_RECURSO','Insumos apontados 1º pre','Dose/ha 1º pre' ,'cd_uso_solo','Data 1º Pre','Pre 1º Operação',]], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], how='left')


#Transformando a coluna 1º Pre em número inteiro
boletim['1º Pre'] = pd.to_numeric(boletim['Pre 1º Operação'],errors='coerce').astype('Int64')

#Definindo o ID do 1º Pre
boletim['1º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['1º Pre'])+'ºControle') if pd.notna(x['1º Pre']) and x['1º Pre'] ==1 else None, axis=1) 

In [544]:
#2º e 3º Controle Pre emergente após plantio


boletim = boletim.merge(id_controle_pre[['CD_BOLETIM_SILVICULTURA','Boletim pre', 'cd_uso_solo','NOME_RECURSO','Pre','Data Pre','Insumos apontados pre','Dose/ha pre']], left_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'],how='left')

#Definindo a data do 2º Pré e o controle

boletim['Data 2º Pre'] = boletim.apply(lambda x: x['Data Pre'] if x['Pre'] == 2 else None,axis=1)

boletim['Insumos apontados 2º Pre'] = boletim.apply(lambda x: x['Insumos apontados pre'] if x['Pre'] == 2 else None,axis=1)

boletim['Dose/ha 2º pre'] = boletim.apply(lambda x: x['Dose/ha pre'] if x['Pre'] == 2 else None,axis=1)


boletim['2º Pre'] = pd.to_numeric(boletim['Pre'],errors='coerce').astype('Int64')

boletim['2º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['2º Pre'])+'ºControle') if pd.notna(x['2º Pre']) and x['2º Pre'] ==2 else None, axis=1) 

#Definindo a data do 3º Pré e o controle

boletim['3º Pre'] = pd.to_numeric(boletim['Pre'],errors='coerce').astype('Int64')

boletim['Data 3º Pre'] = boletim.apply(lambda x: x['Data Pre'] if x['Pre'] == 3 else None, axis= 1)


boletim['Insumos apontados 3º Pre'] = boletim.apply(lambda x: x['Insumos apontados pre'] if x['Pre'] == 3 else None,axis=1)

boletim['Dose/ha 3º pre'] = boletim.apply(lambda x: x['Dose/ha pre'] if x['Pre'] == 3 else None,axis=1)


boletim['3º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['3º Pre'])+'ºControle') if pd.notna(x['3º Pre']) and x['3º Pre'] ==3 else None, axis=1)


#Organizando as datas do Pre
boletim['Pre'] = boletim.apply(lambda x: x['Pre 1º Operação'] if pd.isna(x['Pre']) else x['Pre'], axis=1)
boletim['Data Pre'] = boletim.apply(lambda x: x['Data 1º Pre'] if pd.isna(x['Data Pre']) else x['Data Pre'], axis=1)

In [545]:
#Barra 

boletim = boletim.merge(barra[['CD_BOLETIM_SILVICULTURA','cd_uso_solo','Boletim Barra','NOME_RECURSO','Barra','Data Barra', 'Dose/ha barra','Insumos apontados barra']], left_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], right_on= ['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], how='left')

#Definindo a data da 1º Barra

boletim['Data 1º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 1 else None,axis=1)

boletim['Dose/ha 1º barra'] = boletim.apply(lambda x: x['Dose/ha barra'] if x['Barra'] == 1 else None,axis=1)

boletim['Insumos apontados 1º barra'] = boletim.apply(lambda x: x['Insumos apontados barra'] if x['Barra'] == 1 else None,axis=1)

boletim['1º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['1º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['1º Barra'])+'ºControle') if pd.notna(x['1º Barra']) and x['1º Barra'] ==1 else None, axis=1) 


#Definindo a data da 2º Barra

boletim['Data 2º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 2 else None,axis=1)

boletim['Dose/ha 2º barra'] = boletim.apply(lambda x: x['Dose/ha barra'] if x['Barra'] == 2 else None,axis=1)

boletim['Insumos apontados 2º barra'] = boletim.apply(lambda x: x['Insumos apontados barra'] if x['Barra'] == 2 else None,axis=1)

boletim['2º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['2º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['2º Barra'])+'ºControle') if pd.notna(x['2º Barra']) and x['2º Barra'] ==2 else None, axis=1) 


#Definindo a data da 3º Barra

boletim['Data 3º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 3 else None,axis=1)

boletim['Dose/ha 3º barra'] = boletim.apply(lambda x: x['Dose/ha barra'] if x['Barra'] == 3 else None,axis=1)

boletim['Insumos apontados 3º barra'] = boletim.apply(lambda x: x['Insumos apontados barra'] if x['Barra'] == 3 else None,axis=1)

boletim['3º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['3º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['3º Barra'])+'ºControle') if pd.notna(x['3º Barra']) and x['3º Barra'] ==3 else None, axis=1) 





In [546]:
# Calagem

#Trazer a operação de calagem para o boletim
boletim = boletim.merge(calcario[['CD_BOLETIM_SILVICULTURA','Boletim Calagem','DCR_OPERACAO','Insumo calagem','cd_uso_solo','Sequência Calagem','Data Calagem','Dose/ha acumulada calagem','regra_operacao_calagem','Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação calagem']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo calagem'],how='left')

boletim['Calagem'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + 'Calagem' if pd.notna(x['Data Calagem']) else None, axis=1)


In [547]:
# Adubação de base

#Trazer as operações de adubação de base para o boletim

boletim = boletim.merge(base[['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Boletim Base','Insumo Adubação base','cd_uso_solo','Sequência base','Data Adub. Base','Dose/ha acumulada base','Dose/ha base','regra_operacao_base','Estratégia Base','Insumo base - rec','Dose kg/ha base - rec','Data de recomendação base']] ,left_on=['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo'],right_on=['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo Adubação base','cd_uso_solo'], how='left')

boletim['Adubação de base'] = boletim.apply(
    lambda x: str(x['cd_uso_solo']) + 'Adubação de base' if pd.notna(x['Data Adub. Base']) and x['regra_operacao_base'] != 'Reforço' else
              str(x['cd_uso_solo']) + 'Reforço' if pd.notna(x['Data Adub. Base']) and x['regra_operacao_base'] == 'Reforço' else
              None, axis=1)

In [548]:
#Adubação de cobertura

#trazer a operação de adubação para o boletim

boletim = boletim.merge(adu_cob[['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Boletim Adu cob','cd_uso_solo','Data Adub. cob','Estratégia ajustada','Insumos apontados', 'Dose/ha adu cob','Dose/ha acumulada adu cob','recomendação comp','Sequência Cob','regra_operação','Insumo Cob','Insumo cobertura - rec', 'Dose kg/ha cobertura - rec','Insumo cobertura 2 - rec', 'Dose kg/ha cobertura 2 - rec','Data de recomendação cobertura','NPK complementar','Dose kg/ha complementar - rec','Corretivos complementares','Dose kg/ha corretivos - rec']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo Cob'],how='left').rename({'NPK complementar':'NPK complementar - rec'},axis=1)

boletim['Adu cob'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_operação']) if pd.notna(x['Data Adub. cob']) else None, axis = 1)

In [549]:
#Exportar Boletim
boletim.to_excel('boletim.xlsx',sheet_name='Boletim', index = False)

In [ ]:
#Aderência 

#aderencia = boletim[['cd_uso_solo', 'cd_uso_solo_origem','Data de criação retalhonado','num_ciclo','cd_regime','num_rotacao','objeto de locação','nom_projeto','vlr_area','ID_REGIAO', 'Mapa_Micro','Divulgação_Book','Data entrega SOP Topografia']]

aderencia = boletim[['cd_uso_solo','num_ciclo','cd_regime','num_rotacao','objeto de locação','nom_projeto','vlr_area','ID_REGIAO', 'Mapa_Micro','Divulgação_Book','Data entrega SOP Topografia']]

aderencia = aderencia.drop_duplicates('cd_uso_solo')


pre_1 = "1ºControle"
pre_2 = "2ºControle"
pre_3 = "3ºControle"
barra_1 = "1ºControle"
barra_2 = "2ºControle"
barra_3 = "3ºControle"
calagem = "Calagem"
reforco = "Reforço"
adub_base = "Adubação de base"
cob1 = "Cob 1"
cob2 = "Cob 2"
complementar1 = "Complementar 1"
complementar2 = "Complementar 2"
replantio1 = '1º Replantio'
replantio2 = '2º Replantio'
replantio3 = '3º Replantio'
replantio_apoio = 'replantio'
form_pre_plant_1 = "1º Formiga Pre Plantio"
form_pre_plant_2 = "2º Formiga Pre Plantio"
desseca_1 = "1º Dessecação"
desseca_2 = "2º Dessecação"
savannah_1 = 'Savannah'
preparo_solo_1 = 'Preparo de Solos'




#Criando os Ids para realizar o merge
aderencia['pre_1'] = aderencia['cd_uso_solo'].astype(str) + pre_1
aderencia['pre_2'] = aderencia['cd_uso_solo'].astype(str) + pre_2
aderencia['pre_3'] = aderencia['cd_uso_solo'].astype(str) + pre_3
aderencia['barra_1'] = aderencia['cd_uso_solo'].astype(str) + barra_1
aderencia['barra_2'] = aderencia['cd_uso_solo'].astype(str) + barra_2
aderencia['barra_3'] = aderencia['cd_uso_solo'].astype(str) + barra_3
aderencia['Calagem'] = aderencia['cd_uso_solo'].astype(str)+ calagem
aderencia['adub_base'] = aderencia['cd_uso_solo'].astype(str)+ adub_base
aderencia['adub_base_reforco'] = aderencia['cd_uso_solo'].astype(str)+ reforco
aderencia['Cob1'] = aderencia['cd_uso_solo'].astype(str)+cob1
aderencia['Cob2'] = aderencia['cd_uso_solo'].astype(str)+cob2
aderencia['Complementar 1'] = aderencia['cd_uso_solo'].astype(str)+complementar1
aderencia['Complementar 2'] = aderencia['cd_uso_solo'].astype(str)+complementar2
aderencia['replantio1'] = aderencia['cd_uso_solo'].astype(str) + replantio1
aderencia['replantio2'] = aderencia['cd_uso_solo'].astype(str) + replantio2
aderencia['replantio3'] = aderencia['cd_uso_solo'].astype(str) + replantio3
aderencia['replantio_apoio'] = aderencia['cd_uso_solo'].astype(str) + replantio_apoio
aderencia['form_pre_plant_1'] = aderencia['cd_uso_solo'].astype(str) + form_pre_plant_1
aderencia['form_pre_plant_2'] = aderencia['cd_uso_solo'].astype(str) + form_pre_plant_2
aderencia['desseca_1'] = aderencia['cd_uso_solo'].astype(str) + desseca_1
aderencia['desseca_2'] = aderencia['cd_uso_solo'].astype(str) + desseca_2
aderencia['savannah_1'] = aderencia['cd_uso_solo'].astype(str) + savannah_1
aderencia['preparo_solo_1'] = aderencia['cd_uso_solo'].astype(str) + preparo_solo_1





#Retalhonamento
#realizar merge com a base de retalhonamento, desde que ela não tenha talhões duplicados

aderencia = aderencia.merge(retalhonamentos[['cd_uso_origem','cd_uso_solo_retalhonamento','Data de criação retalhonado']], left_on='cd_uso_solo', right_on='cd_uso_solo_retalhonamento', how='left')#.drop('cd_uso_retalhonamento',axis=1)


#retalhonamentos['cd_uso_solo_retalhonado'


#1º Desseca
aderencia = aderencia.merge(boletim[['1º Dessecação', 'DCR_OPERACAO', 'Data 1º Dessecação','NOME_RECURSO', 'Dose/ha desseca', 'Dose/ha desseca acumulada', 'Boletim Dessecação']], left_on = 'desseca_1' , right_on= '1º Dessecação', how='left')
aderencia = aderencia.rename({'Dose/ha desseca':'Dose/ha 1º Desseca', 'Dose/ha desseca acumulada':'Dose/ha 1º desseca acumulada','NOME_RECURSO' : 'Insumo 1º Dessecação', 'Boletim Dessecação' : 'Boletim 1º Dessecação' , 'DCR_OPERACAO' : 'Operação 1º Desseca'}, axis = 1)


#2º Desseca
aderencia = aderencia.merge(boletim[['2º Dessecação', 'DCR_OPERACAO', 'Data 2º Dessecação','NOME_RECURSO', 'Dose/ha desseca', 'Dose/ha desseca acumulada', 'Boletim Dessecação']], left_on = 'desseca_2' , right_on= '2º Dessecação', how='left')
aderencia = aderencia.rename({'Dose/ha desseca':'Dose/ha 2º Desseca', 'Dose/ha desseca acumulada':'Dose/ha 2º desseca acumulada', 'NOME_RECURSO' : 'Insumo 2º Dessecação','Boletim Dessecação' : 'Boletim 2º Dessecação' , 'DCR_OPERACAO' : 'Operação 2º Desseca'}, axis = 1)


#1º Formiga Pre Plantio
aderencia = aderencia.merge(boletim[['1º Formiga Pre Plantio','DCR_OPERACAO','Data 1º Formiga Pre Plantio','Dose acumulada/ha form pre plantio', 'boletim formiga pre plantio']], left_on='form_pre_plant_1', right_on='1º Formiga Pre Plantio', how = 'left')
aderencia = aderencia.rename({'Dose acumulada/ha form pre plantio': 'Dose acumulada/ha  1º Formiga Pre Plantio' , 'DCR_OPERACAO': 'Operação 1º Formiga Pre Plantio', 'boletim formiga pre plantio':'Boletim 1º Formiga Pre Plantio'}, axis=1)


#2º Formiga Pre Plantio
aderencia = aderencia.merge(boletim[['2º Formiga Pre Plantio','DCR_OPERACAO','Data 2º Formiga Pre Plantio','Dose acumulada/ha form pre plantio', 'boletim formiga pre plantio']], left_on='form_pre_plant_2', right_on='2º Formiga Pre Plantio', how = 'left')
aderencia = aderencia.rename({'Dose acumulada/ha form pre plantio': 'Dose acumulada/ha  2º Formiga Pre Plantio' , 'DCR_OPERACAO': 'Operação 2º Formiga Pre Plantio', 'boletim formiga pre plantio':'Boletim 2º Formiga Pre Plantio'}, axis=1)


#Savannah
aderencia = aderencia.merge(boletim[['Savannah','Data Savannah', 'Boletim Savannah']], left_on = 'savannah_1' , right_on= 'Savannah',how = 'left')


#Preparo de solos
aderencia = aderencia.merge(boletim[['Preparo de solos', 'Data Preparo de Solos','Boletim Preparo de Solos']], left_on= 'preparo_solo_1', right_on='Preparo de solos', how = 'left')



#Plantio
aderencia = aderencia.merge(data_de_plantio[['Data de Plantio','cd_uso_solo','Material Genético Plantado','Classe de Material Genético Plantado']], left_on= 'cd_uso_solo', right_on='cd_uso_solo', how='left')


#1º Replantio 
aderencia = aderencia.merge(boletim[['Data 1º Replantio','DCR_OPERACAO','Boletim Replantio','1º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo','Classe MG Plantio','Material Genético','VLR_PRODUCAO']] , left_on = 'replantio1', right_on = '1º Replantio', how = 'left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 1º Replantio','Classe MG Plantio':'Classe MG Plantio - 1º Replantio', 'Material Genético': 'Material Genético - 1º Replantio', 'VLR_PRODUCAO':'Valor de produção - 1º Replantio'},axis=1)
aderencia['Classe MG Plantado vs 1 º Replantio '] = aderencia.apply(lambda x: str(x['Classe de Material Genético Plantado']) + " - " + str(x['Classe MG Plantio - 1º Replantio']) if  pd.notna(x['Classe MG Plantio - 1º Replantio']) and pd.notna(x['Material Genético Plantado']) else x['Classe de Material Genético Plantado'], axis=1)


#2º Replantio 
aderencia = aderencia.merge(boletim[['Data 2º Replantio','DCR_OPERACAO','Boletim Replantio','2º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo','Classe MG Plantio','Material Genético', 'VLR_PRODUCAO']] , left_on = 'replantio2', right_on = '2º Replantio', how = 'left', suffixes = ('', ' 2º Replantio'))
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 2º Replantio','Classe MG Plantio':'Classe MG Plantio - 2º Replantio', 'Material Genético': 'Material Genético - 2º Replantio','VLR_PRODUCAO':'Valor de produção - 2º Replantio'},axis=1)
aderencia['Classe MG Plantado vs 2 º Replantio '] = aderencia.apply(lambda x: str(x['Classe de Material Genético Plantado']) + " - " + str(x['Classe MG Plantio - 2º Replantio']) if  pd.notna(x['Classe MG Plantio - 2º Replantio']) and pd.notna(x['Material Genético Plantado']) else x['Classe de Material Genético Plantado'], axis=1)


#3º Replantio 
aderencia = aderencia.merge(boletim[['Data 3º Replantio','DCR_OPERACAO','Boletim Replantio','3º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo','Classe MG Plantio','Material Genético','VLR_PRODUCAO']] , left_on = 'replantio3', right_on = '3º Replantio', how = 'left', suffixes = ('', ' 3º Replantio'))
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 3º Replantio','Classe MG Plantio':'Classe MG Plantio - 3º Replantio','Material Genético': 'Material Genético - 3º Replantio','VLR_PRODUCAO':'Valor de produção - 3º Replantio'},axis=1)
aderencia['Classe MG Plantado vs 3 º Replantio '] = aderencia.apply(lambda x: str(x['Classe de Material Genético Plantado']) + " - " + str(x['Classe MG Plantio - 3º Replantio']) if  pd.notna(x['Classe MG Plantio - 3º Replantio']) and pd.notna(x['Material Genético Plantado']) else x['Classe de Material Genético Plantado'], axis=1)

aderencia['Classe MG Plantado vs Replantio'] = (
    aderencia.apply(
        lambda x: f"{str(x['Classe de Material Genético Plantado'])} - {str(x['Classe MG Plantio - 1º Replantio'])} - {str(x['Classe MG Plantio - 2º Replantio'])} - {str(x['Classe MG Plantio - 3º Replantio'])}"
        if pd.notna(x['Classe de Material Genético Plantado']) else None, axis=1
    )
    .str.replace('- nan', '')
    .str.replace('nan - ', '')
    .str.replace('nan', '')
)



#Replantio apoio 
aderencia = aderencia.merge(boletim[['Replantio_apoio','numero de replantios']].drop_duplicates('Replantio_apoio'), left_on = 'replantio_apoio', right_on = 'Replantio_apoio', how = 'left')


#Preparo refeência pre emergente
aderencia = aderencia.merge(data_de_preparo[['Data Preparo referência pre','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')


#1º Pre
aderencia = aderencia.merge(boletim[['Data 1º Pre','DCR_OPERACAO','1º Pre','Insumos apontados 1º pre', 'Dose/ha 1º pre','Boletim 1º pre']], left_on='pre_1', right_on='1º Pre',how='left').drop('1º Pre',axis=1)
aderencia = aderencia.rename({'DCR_OPERACAO': 'Operação 1º Pre'},axis=1)


#2º Pre
aderencia = aderencia.merge(boletim[['Data 2º Pre','DCR_OPERACAO','2º Pre','Insumos apontados 2º Pre','Dose/ha 2º pre','Boletim pre']], left_on='pre_2', right_on='2º Pre',how='left').drop('2º Pre',axis=1)
aderencia = aderencia.rename({'Boletim pre':'Boletim 2º Pre','DCR_OPERACAO': 'Operação 2º Pre'}, axis=1)


#3º Pre
aderencia = aderencia.merge(boletim[['Data 3º Pre','DCR_OPERACAO','3º Pre','Insumos apontados 3º Pre','Dose/ha 3º pre','Boletim pre']], left_on='pre_3', right_on='3º Pre',how='left').drop('3º Pre',axis=1)
aderencia = aderencia.rename({'Boletim pre':'Boletim 3º Pre','DCR_OPERACAO': 'Operação 3º Pre' }, axis=1)


#1º Barra
aderencia = aderencia.merge(boletim[['Data 1º Barra','DCR_OPERACAO','1º Barra' ,'Dose/ha 1º barra','Insumos apontados 1º barra','Boletim Barra']], left_on= 'barra_1', right_on='1º Barra',how='left').drop('1º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 1º Barra','DCR_OPERACAO':'Operação 1º Barra' }, axis=1)


#2º Barra
aderencia = aderencia.merge(boletim[['Data 2º Barra','DCR_OPERACAO','2º Barra','Dose/ha 2º barra','Insumos apontados 2º barra','Boletim Barra']], left_on= 'barra_2', right_on='2º Barra',how='left').drop('2º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 2º Barra','DCR_OPERACAO':'Operação 2º Barra'}, axis=1)


#3º Barra
aderencia = aderencia.merge(boletim[['Data 3º Barra','DCR_OPERACAO','3º Barra','Dose/ha 3º barra','Insumos apontados 3º barra','Boletim Barra']], left_on= 'barra_3', right_on='3º Barra',how='left').drop('3º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 3º Barra','DCR_OPERACAO':'Operação 3º Barra'}, axis=1)


#Calagem
aderencia = aderencia.merge(boletim[['Data Calagem','DCR_OPERACAO','Boletim Calagem','Dose/ha acumulada calagem','Calagem','Insumo calagem','Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação calagem']], left_on='Calagem', right_on='Calagem', how='left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação Calagem'},axis= 1)


#Adubação de base
aderencia = aderencia.merge(boletim[['Data Adub. Base','DCR_OPERACAO','Boletim Base','Dose/ha acumulada base', 'Adubação de base','Insumo Adubação base','Estratégia','Insumo base - rec','Dose kg/ha base - rec','Data de recomendação base']] ,right_on='Adubação de base', left_on='adub_base', how = 'left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação Base'},axis=1)


#Adubação de reforço
aderencia = aderencia.merge(boletim[['Data Adub. Base','DCR_OPERACAO','Boletim Base','Dose/ha base', 'Adubação de base','Insumo Adubação base']] ,right_on='Adubação de base', left_on='adub_base_reforco', how = 'left', suffixes=('', ' - Reforço'))
aderencia= aderencia.rename({'Dose/ha base':'Dose/ha base - Reforço','DCR_OPERACAO':'Operação de Reforço'}, axis=1)


#Cob 1
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'Adu cob','Insumo cobertura - rec','Dose kg/ha cobertura - rec','Data de recomendação cobertura']], right_on='Adu cob', left_on='Cob1', how= 'left').rename({'Data de recomendação cobertura':'Data de recomendação cobertura 1'},axis=1)
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Cob 1','Data Adub. cob':'Data Adub. cob - Cob 1','DCR_OPERACAO':'Operação Cob 1'}, axis=1)


#Cob 2
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'Adu cob','Insumo cobertura 2 - rec','Dose kg/ha cobertura 2 - rec','Data de recomendação cobertura']], right_on='Adu cob', left_on='Cob2', how= 'left',suffixes=('', ' - Cob 2')).rename({'Data de recomendação cobertura':'Data de recomendação cobertura 2'},axis=1)
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Cob 2','Data Adub. cob':'Data Adub. cob - Cob 2' , 'DCR_OPERACAO':'Operação Cob 2'}, axis=1)


#Complementar 1
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'NPK complementar - rec' ,'Dose kg/ha complementar - rec','recomendação comp', 'Adu cob','Corretivos complementares' , 'Dose kg/ha corretivos - rec' ]], right_on='Adu cob', left_on='Complementar 1', how= 'left',suffixes=('', ' - Complementar 1'))
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Complementar 1','Data Adub. cob':'Data Adub. cob - Complementar 1','DCR_OPERACAO':'Operação complementar 1', 'Corretivos complementares' : 'Corretivos complementares 1', 'Dose kg/ha corretivos - rec' : 'Dose kg/ha corretivos - rec 1' }, axis=1)


#Complementar 2
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob','Corretivos complementares','Dose kg/ha corretivos - rec', 'Adu cob']], right_on='Adu cob', left_on='Complementar 2', how= 'left',suffixes=('', ' - Complementar 2'))
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Complementar 2','Data Adub. cob':'Data Adub. cob - Complementar 2', 'DCR_OPERACAO':'Operação complementar 2','Corretivos complementares' : 'Corretivos complementares 2', 'Dose kg/ha corretivos - rec' : 'Dose kg/ha corretivos - rec 2' }, axis=1)


#Renomear as operações
aderencia = aderencia.rename({ 
'mudas/ha': 'mudas/ha 1º Replantio', 
'mudas/ha recomendado' : 'mudas/ha recomendado 1º Replantio' ,
'Replantio Efetivo' : 'Replantio Efetivo 1º Replantio' , 
'Boletim Replantio': 'Boletim Replantio - 1º Replantio','Data de criação retalhonado' : 'Data de retalhonamento',
'Mapa_Micro': 'Data Mapa Micro', 'Divulgação_Book':'Data Divulgação Book'
},axis=1)


# Criar uma máscara para identificar linhas onde todas as colunas são 'NaN'
colunas_nulas= ['Data de Plantio','Data 1º Pre', 'Data 2º Pre', 'Data 3º Pre', 'Data 1º Barra', 'Data 2º Barra','Data 3º Barra', 'Data Calagem' , 'Data Adub. Base', 
'Data Adub. Base - Reforço', 'Data Adub. cob - Cob 1','Data Adub. cob - Cob 2','Data Adub. cob - Complementar 1','Data Adub. cob - Complementar 2',
'Data 1º Replantio' ,'Data 2º Replantio','Data 3º Replantio' , 'Data 1º Formiga Pre Plantio' , 'Data 2º Formiga Pre Plantio','Data 1º Dessecação', 'Data 2º Dessecação','Data Savannah','Data Preparo de Solos']

aderencia_pre_emergente_mascara = aderencia[colunas_nulas].isna().all(axis=1)

#Excluindo somente  as colunas que não são nulas
aderencia = aderencia[~aderencia_pre_emergente_mascara]


aderencia = aderencia.drop(['ID_REGIAO','pre_1','pre_2','pre_3','barra_1','barra_2','barra_3','Calagem','adub_base',
'Adubação de base','adub_base_reforco' , 'Adubação de base - Reforço','Adu cob','Adu cob - Cob 2','Adu cob - Complementar 1',
'Adu cob - Complementar 2','Cob1', 'Cob2','Complementar 1','Complementar 2','1º Replantio','2º Replantio','3º Replantio',
'replantio1','replantio2','replantio3','replantio_apoio','Replantio_apoio', 'form_pre_plant_1', 'form_pre_plant_2',
'1º Formiga Pre Plantio','2º Formiga Pre Plantio', 'desseca_1', '1º Dessecação', 'desseca_2', '2º Dessecação','savannah_1', 'Savannah','Preparo de solos', 'preparo_solo_1' ],axis=1)

In [ ]:
#Definindo mês operacional das operações
#aderencia = mes_operacional(aderencia,'Data de Preparo','Mês Operacional Preparo')
#aderencia = mes_operacional(aderencia,'Data de Plantio','Mês Operacional Plantio')
#aderencia = mes_operacional(aderencia,'Data 1º Pre','Mês Operacional 1º Pre')
#aderencia = mes_operacional(aderencia,'Data 2º Pre','Mês Operacional 2º Pre')
#aderencia = mes_operacional(aderencia,'Data 3º Pre','Mês Operacional 3º Pre')
#aderencia = mes_operacional(aderencia,'Data 1º Barra','Mês Operacional 1º Barra')
#aderencia = mes_operacional(aderencia,'Data 2º Barra','Mês Operacional 2º Barra')


In [ ]:
#Exportar aderência
aderencia.to_excel('aderencia.xlsx',index = False,sheet_name='aderencia')

In [ ]:
#Log de premissas

dpct_aderencia = aderencia[aderencia.cd_uso_solo.duplicated()].shape[0]
hora_fim = datetime.today()
tempo_de_execucao = hora_fim - hora_inicio
last_note = boletim['DATA_OPERACAO'].max().strftime('%d/%m/%y')
hora_exe = datetime.now().strftime('%H:%M:%S')

cobertura_log = aderencia[['objeto de locação', 'Data Adub. cob - Cob 1' , 'Data Adub. cob - Cob 2']]
cobertura_log = cobertura_log[(pd.notna(cobertura_log['Data Adub. cob - Cob 2'])) & (pd.isna(cobertura_log['Data Adub. cob - Cob 1']))]



log_premissas = 'Log de premissas.txt'

l = '_'*100

#Ordenando em ordem alfabética

delta_insumo = sorted(delta_insumo)

delta_operacao = sorted(delta_operacao)

sem_cadastro_mg = sorted(sem_cadastro_mg)


with open (log_premissas, 'w', encoding='latin-1') as arquivo:

    #Lista de insumos não cadastrados
    arquivo.write('Lista de Insumos que não estão cadastrados em premissas:' + '\n')

    for i in delta_insumo:
        arquivo.write(str(i)+'\n')

    arquivo.write('\n' + l + '\n')

    #Lista de operações
    arquivo.write('\n'+'Lista de Operações que não estão cadastradas em premissas:' +'\n')
    
    for o in delta_operacao:
        arquivo.write(str(o)+'\n')
    arquivo.write('\n' + l + '\n')


    arquivo.write('\n' + 'Lista de Talhões que tem apontamento de Cob2 mas não tem Cob1:' + '\n')

    for k,v in cobertura_log.iterrows():
        arquivo.write(str(v['objeto de locação']) + '\n')
    arquivo.write('\n' + l + '\n')


    #lista de material genéticos que não estão cadastrados em premissas
    arquivo.write('\n'+'Lista de materiais genéticos que não estão cadastrados em premissas:'+'\n')
    
    for mg in sem_cadastro_mg:
        arquivo.write(str(mg)+'\n')
    arquivo.write('\n' + l + '\n') 




    arquivo.write(f"""
    Resumo de Execução:                
                    
                    
    O último apontamento do boletim é de: {last_note};

    Temos {dpct_aderencia} talhões duplicados na base de Aderência;

    Executado e arquivo txt de Log de premissas gerado com sucesso ;

    O tempo de execução foi de {round(tempo_de_execucao.total_seconds()/60)} minutos.

    Executado as {hora_exe} hrs.
    """)
